In [0]:

import plotly.express as px
import pandas as pd
import numpy as np
from scipy.stats import poisson
import datetime as dt
from scipy import stats
pd.set_option('display.max_columns', None)

In [0]:
def L1_causality(df, result, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, L1KPI, L1Num, L1Den, L2KPI1, L2Num1, L2Den1, L2KPI2, L2Num2, L2Den2, L2KPI3, L2Num3, L2Den3, L2KPI4, L2Num4, L2Den4, an_date, an_item1, an_item2, an_item3):
    
    #data is defined as the dataset used in the preceeding Anomaly Detection run
    data = df

    #KPI's are defined and data is cleaned
    data = data.replace('#VALUE!', 0)
    data = data.replace('#DIV/0!', 0)
    data[L1KPI] = data[L1Num]/data[L1Den]
    data[L2KPI1] = data[L2Num1]/data[L2Den1]
    data[L2KPI2] = data[L2Num2]/data[L2Den2]
    data[L2KPI3] = data[L2Num3]/data[L2Den3]
    data[L2KPI4] = data[L2Num4]/data[L2Den4]
    data = data.replace(np.inf, 0)
    data = data.fillna(0)
    data[f'{L2KPI4}'] = data[f'{L2KPI4}'].astype(float)
    data[f'{L2KPI3}'] = data[f'{L2KPI3}'].astype(float)
    data[f'{L2KPI2}'] = data[f'{L2KPI2}'].astype(float)
    data['Par_Basis'] = data[L2KPI2] * data[L2KPI1] / data[L2KPI3]
    
    data = data.sort_values(by=['timestamp'])    
    
    timelist = list(range(0,baseline_window,1))
    timeArray = np.array(timelist)
    
    group = list(data[field1].unique())
    groups = []
    for c in group:
        locals()[c] = data.loc[data[field1] == c]
        groups.append(locals()[c])
        
    #Iterates over each unique item in the field1 column and calculates trends for each KPI
    for i in groups:
      
      i[f'Avg_{L2KPI1}'] = i[L2KPI1].rolling(window = baseline_window).mean()
      i[f'Avg_{L2KPI2}'] = i[f'{L2KPI2}'].rolling(window = baseline_window).mean()
      i[f'Avg_{L2KPI3}'] = i[f'{L2KPI3}'].rolling(window = baseline_window).mean()
      i[f'Avg_{L2KPI4}'] = i[f'{L2KPI4}'].rolling(window = baseline_window).mean()
      
      #Trend is calculated for each KPI
      Array = i[L1KPI].to_numpy()
      Trend = []
      for r in range(0, len(Array), 1):
          if r>(baseline_window-2):
              x = timeArray
              y = Array[timeArray-(baseline_window-1)+r]
              m, b = np.polyfit(x, y, 1)
              Trend.append(m)
          else:
              Trend.append(0)
      Trend = np.asarray(Trend)
      i[f'{L1KPI}_Trend'] = Trend


      Array = i[L2KPI1].to_numpy()
      Trend = []
      for r in range(0, len(Array), 1):
          if r>(baseline_window-2):
              x = timeArray
              y = Array[timeArray-(baseline_window-1)+r]
              m, b = np.polyfit(x, y, 1)
              Trend.append(m)
          else:
              Trend.append(0)
      Trend = np.asarray(Trend)
      i[f'{L2KPI1}_Trend'] = Trend
      i[f'{L2KPI2}_{L2KPI1}'] = i[f'Avg_{L2KPI2}'] / i[f'Avg_{L2KPI3}']
      i[f'Par_{L2KPI1}'] = i[f'{L2KPI2}_{L2KPI1}'] * i[f'{L2KPI4}']
      i[f'Par_{L2KPI4}'] = i[f'{L2KPI2}_{L2KPI1}'] / i[f'Avg_{L2KPI3}']


      Array = i[L2KPI2].to_numpy()
      Trend = []
      for r in range(0, len(Array), 1):
          if r>(baseline_window-2):
              x = timeArray
              y = Array[timeArray-(baseline_window-1)+r]
              m, b = np.polyfit(x, y, 1)
              Trend.append(m)
          else:
              Trend.append(0)

      Trend = np.asarray(Trend)
      i[f'{L2KPI2}_Trend'] = Trend
      i[f'{L2KPI1}_{L2KPI3}']= i[f'Avg_{L2KPI1}'] / i[f'Avg_{L2KPI3}'] 
      i[f'Par_{L2KPI2}'] = i[f'{L2KPI1}_{L2KPI3}'] * i[f'{L2KPI4}']


      Array = i[L2KPI3].to_numpy()
      Trend = []
      for r in range(0, len(Array), 1):
          if r>(baseline_window-2):
              x = timeArray
              y = Array[timeArray-(baseline_window-1)+r]
              m, b = np.polyfit(x, y, 1)
              Trend.append(m)
          else:
              Trend.append(0)

      
      Trend = np.asarray(Trend)
      i[f'{L2KPI3}_Trend'] = Trend
      i[f'Frq_Ser_{L2KPI3}']= - (i[f'Avg_{L2KPI2}'] * i[f'Avg_{L2KPI1}'] * i[f'Avg_{L2KPI4}'])
      i[f'Sq_{L2KPI3}'] = i[f'Avg_{L2KPI3}'] * i[f'Avg_{L2KPI3}'] 
      i[f'Par_{L2KPI3}'] = i[f'Frq_Ser_{L2KPI3}'] / i[f'Sq_{L2KPI3}']
      i[f'Avg_{L1KPI}'] = i[f'{L1KPI}'].mean()

      Array = i[f'{L2KPI4}'].to_numpy()
      Trend = []
      for r in range(0, len(Array), 1):
          if r>(baseline_window-2):
              x = timeArray
              y = Array[timeArray-(baseline_window-1)+r]
              m, b = np.polyfit(x, y, 1)
              Trend.append(m)
          else:
              Trend.append(0)

      Trend = np.asarray(Trend)
      i[f'{L2KPI4}_Trend'] = Trend 
      i[f'Par_{L2KPI4}'] = i[f'{L2KPI2}_{L2KPI1}'] / i[f'Avg_{L2KPI3}']
    
    data = pd.concat(groups).fillna(0)
    #Contribution metrics
    
    
    data[f'{L2KPI3}_cont'] = data[f'{L2KPI3}_Trend'] * data[f'Par_{L2KPI3}']
    data[f'{L2KPI1}_cont'] = data[f'{L2KPI1}_Trend'] * data[f'Par_{L2KPI1}']
    data[f'{L2KPI2}_cont'] = data[f'{L2KPI2}_Trend'] * data[f'Par_{L2KPI2}']
    data['Basis_cont'] = data[f'{L2KPI4}_Trend'] * data[f'Par_{L2KPI4}']
    data[f'{L2KPI3}_%{L1KPI}'] = data[f'{L2KPI3}_cont'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI1}_%{L1KPI}'] = data[f'{L2KPI1}_cont'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI2}_%{L1KPI}'] = data[f'{L2KPI2}_cont'] / data[f'Avg_{L1KPI}']
    data[f'Basis_%{L1KPI}'] = data['Basis_cont'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI3}_atr'] = data[f'{L2KPI3}_cont'] / data[f'{L1KPI}_Trend']
    data[f'{L2KPI1}_atr'] = data[f'{L2KPI1}_cont'] / data[f'{L1KPI}_Trend']
    data[f'{L2KPI2}_atr'] = data[f'{L2KPI2}_cont'] / data[f'{L1KPI}_Trend']
    data['Basis_atr'] = data['Basis_cont'] / data[f'{L1KPI}_Trend']
    data[f'{L1KPI}_atr'] = data[f'{L2KPI3}_atr'] + data[f'{L2KPI1}_atr'] + data[f'{L2KPI2}_atr']

    #Some final data cleaning and the calculation of the premium score by multiplying Trend and ULT_EARNED_PREMIUM
    #Currently quoted out but also the ability to filter by date if the user wanted to analyze a single anomaly point

    data[f'{L1KPI}_Trend'] = data[f'{L1KPI}_Trend'].astype(float)
    data[f'{L2KPI3}'] = data[f'{L2KPI3}'].astype(float)
    data.fillna(0)

    #Filters the dataset to only show the outliers that are imported from the 'result' file and only
    #In order to calculate accurate trend values I have to first analyze the full dataset and filter to show outliers after

    results = data.loc[data['timestamp'] == an_date]
    global kpiTrend
    if an_item3 != '':
      kpiTrend = results[f'{kpi1}_Trend'].loc[(results['timestamp'] == an_date) & (results['combo'] == (f'{an_item1},{an_item2},{an_item3}'))]
      kpiTrend = kpiTrend.iloc[0]
            
    elif an_item2 !='':
      kpiTrend = results[f'{kpi1}_Trend'].loc[(results['timestamp'] == an_date) & (results['combo'] == (f'{an_item1},{an_item2}'))]
      kpiTrend = kpiTrend.iloc[0]
      
    else:
      kpiTrend = results[f'{kpi1}_Trend'].loc[(results['timestamp'] == an_date) & (results['combo'] == an_item1)]
      kpiTrend = kpiTrend.iloc[0]
      
    results = results[['timestamp', field1, f'{kpi1}_Trend', f'{L2KPI3}_cont',f'{L2KPI1}_cont', f'{L2KPI2}_cont', f'{L2KPI3}_%ALR', f'{L2KPI1}_%ALR', f'{L2KPI2}_%ALR', f'{L2KPI3}_atr', f'{L2KPI1}_atr', f'{L2KPI2}_atr', f'{L1KPI}_atr']]
    #results.to_csv('Data/CausalityTest.csv', header = True, index = False)
    display(results)
    return(results)

In [0]:
def L21_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend, l1_items, ranking_field, ranking_function, bins_cols, bins_size, date_field):
  import glob
  import os
  from datetime import datetime, timedelta
  #import pandasql as ps
  from pyspark.sql import SQLContext
  import pandas as pd
  pd.options.mode.chained_assignment = None
  
  sqlContext = SQLContext(sc)
  import time

  start = time.time()
  print("file read from dbfs started")
  

  #file path
  #path = '/dbfs/sg_claims_files/'
  path = file_path
  # check if string "claims" is part of the filepath - to identify claims/loss ratio database
  if 'claims' in path:
    #Read part train files from DBFS and concatenate into dataframe
    all_files = glob.glob(os.path.join(path, "SG_Claims_Training_0.ftr"))     
    df_from_each_file = (pd.read_feather(f) for f in all_files) #all columns
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    # calcualte claims count and policy_cvog count from raw data (since these fields are required to calculate some KPI's it has been hardcoded. Assumption is that these fields will be already available in the data later)
    concatenated_df['CLAIMS_COUNT']= np.where((concatenated_df['ULT_CLAIM_INCURRED'])>0, concatenated_df['CLAIMNUMBER'] + concatenated_df['COVERAGE_GROUP'] , '999')
    concatenated_df['POL_CVG_COUNT']= np.where((concatenated_df['POLICY']).notna(), concatenated_df['POLICY'] + concatenated_df['COVERAGE_GROUP'] , '999')
  #for billing and other database files which are single files in .csv format  
  else:
    #read file
    concatenated_df = pd.read_csv(path)
  
  #Convert date_field to pandas datetime
  concatenated_df[date_field] = pd.to_datetime(concatenated_df[date_field])
  # Date Filter
  concatenated_df = concatenated_df.loc[(concatenated_df[date_field] >= start_date) & (concatenated_df[date_field] <= end_date)]
  
  #Binning columns with user selected column names and bin sizes
  for c in range(len(bins_cols)):
    concatenated_df[bins_cols[c]+'_bins'] = pd.qcut(concatenated_df[bins_cols[c]], q=bins_size[c], precision = 0).astype(str)
  
  #Add month and year columns
  concatenated_df['Accd_month'] = pd.to_datetime(concatenated_df[date_field]).dt.month_name()
  concatenated_df['Accd_year'] = pd.to_datetime(concatenated_df[date_field]).dt.year
  concatenated_df['Accd_quarter'] = pd.to_datetime(concatenated_df[date_field]).dt.quarter
  
  end = time.time()
  print("file read from dbfs ended",end - start)
  
  #Filter data on values in the anomaly result case as per the fields selected in Anomaly Detection. 
  if((field1 !="") & (field2!="") & (field3!="")):
       concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2) & ( concatenated_df[field3] == an_item3)]
      
      #display(df)
  
  elif((field1 !="")   & (field2=="") & (field3=="")): 
      concatenated_df = concatenated_df.loc[concatenated_df[field1] == an_item1]
      
      #display(concatenated_df)
  
  elif((field1 !="")   & (field2 !="" ) & (field3=="")):
     concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2)]
    
    #display(concatenated_df)
  
  #l1 data preprocessing------------------------------------------------------------------------
  #Filter on level 1 field
  
  
  #data = concatenated_df
  #put aggregation columns in a list
  lst1 = [level1]
  #put columns for year and month in list
  if(period == 'month'):
      segment = ['Accd_year','Accd_month']
      
    #put columns for year and quarter in list
  if(period == 'quarter'):
      segment = ['Accd_year','Accd_quarter']
      
  segment.extend(lst1)
  
  if(level1!=''):


    # calculate KPI values for KPIs selected by user
      
    data = concatenated_df.groupby(segment).agg({L1Num: num_function_1 , L1Den: den_function_1, ranking_field: ranking_function })
    #data = concatenated_df.groupby(segment)
    data.reset_index(inplace = True)
    #display(data)
    #if empty data set obtained after aggregation
    if data.empty:
      return('No records found for this selection!')
    #Calculate values for KPI's for each row timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    data[L1KPI] = data[L1Num]/data[L1Den]
    #Create timestamp column
    if(period == 'month'):
      data['Month Name-Year']=pd.to_datetime(data['Accd_month']+data['Accd_year'].astype(str),format='%B%Y')
      data['timestamp'] = data['Month Name-Year'].dt.strftime('%Y-%m')

    if(period == 'quarter'):
      data['Accd_quarter']=data['Accd_quarter'].replace({1:'1', 2: '4', 3: '7', 4: '10'})
      data['quart_year']=pd.to_datetime(data['Accd_year'].astype(str) +data['Accd_quarter'].astype(str),format='%Y%m')
      data['timestamp'] = data['quart_year'].dt.strftime('%Y-%m')
      

    #Create a list of the dates within the rolling window used for causal analysis
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data =  data.sort_values(by='timestamp', ascending=True)
    #Replace invalid values
    data = data.replace('#DIV/0!', np.NaN)
    data = data.replace('#VALUE!', np.NaN)
    #Conversion to float
    data[L1KPI] = data[L1KPI].astype(str).astype(float)
    numeric_columns = data.select_dtypes(include=['number']).columns
    from dateutil.relativedelta import relativedelta
  
    if(period == 'month'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,baseline_window,1))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=baseline_window - 1 - i)
        timelist.append( datetime(past_date.year, past_date.month, 1).strftime('%Y-%m-%d') )
      print(timelist)    
      timeArray = np.array(timelist)
      
    if(period == 'quarter'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,(baseline_window*3),3))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=(baseline_window*3) - 3 - i)
        past_date = past_date.strftime('%Y-%m-%d')
        timelist.append(past_date)
      print(timelist)  
      timeArray = np.array(timelist)
 
    #create a separate dataframe for each date in the rolling window
    dgroups = []
    for d in timelist:
        locals()[d] = data.loc[data['timestamp'] == d]
        dgroups.append(locals()[d])
    data = pd.concat(dgroups).fillna(0)
    
    #timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    timelist = list(range(0,baseline_window,1))
    timeArray = np.array(timelist)
    len(timeArray)
    data = data.fillna(0)

    #Create a dataframe for each unique value in the level1 column
    group = list(data[level1].unique())
    groups = []
    for c in group:
        locals()[c] = data.loc[data[level1] == c]
        groups.append(locals()[c])

    for s in groups:
        Array = s[L1KPI].to_numpy()
        Trend = []
        for i in range(0, len(Array), 1):
            if i>(baseline_window-2):
                x = timeArray
                y = Array[timeArray-(baseline_window-1)+i]
                m, b = np.polyfit(x, y, 1)
                Trend.append(m)
            else:
                Trend.append(0)

        Trend = np.asarray(Trend)
        s[f'{L1KPI}_Trend'] = Trend

        #Weighting basis
        s[f'Avg_{L1KPI}'] = s[f'{L1KPI}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L1Num}'] = s[f'{L1Num}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L1Den}'] = s[f'{L1Den}'].rolling(window = baseline_window).mean()

    data = pd.concat(groups).fillna(0)

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    #Loops over each rolling window and calculates the weights for each occurance
    for s in dates:
        Array = s[f'Avg_{L1Den}'].to_numpy()
        s[f'{L1KPI}_weight_basis'] = Array.sum()
        
    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Weight'] = data[f'Avg_{L1Den}'] / data[f'{L1KPI}_weight_basis']

    #Calculates aggregated weights to ensure it adds up to exactly 1
    #data1 = data.loc[data['timestamp'] == an_date]
    #data1[f'Agg_Weight'] = data1[f'{L1KPI}_Weight'].sum()

    data[f'{L1KPI}_Slope_%ofAvg'] = data[f'{L1KPI}_Trend'] / data[f'Avg_{L1KPI}']
    data[f'{L1KPI}_Trend_Decomp'] = data[f'{L1KPI}_Weight'] * data[f'{L1KPI}_Trend']

    #Loops over each rolling window to calculate trend decomp and business mix
    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'Agg_{L1KPI}_Trend_Decomp'] = s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'{L1KPI}_Business_Mix'] = kpiTrend - s[f'{L1KPI}_Trend_Decomp'].sum()

    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Attribution'] = data[f'{L1KPI}_Trend_Decomp'] / kpiTrend
    data[f'{L1KPI}_Impact_%{L1KPI}'] = data[f'{L1KPI}_Trend_Decomp'] / data[f'Avg_{L1KPI}']

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'{L1KPI}_Att_Business_Mix'] = s[f'{L1KPI}_Business_Mix'] / kpiTrend
        s[f'{L1KPI}_Att_Agg'] = s[f'{L1KPI}_Attribution'].sum() + s[f'{L1KPI}_Att_Business_Mix']
    data = pd.concat(dates).fillna(0)   
    data[f'Agg_Trend_Decomp'] = data[f'{L1KPI}_Trend_Decomp'].sum()
    result1 = data.loc[data['timestamp'] == an_date]
    
    
    #if level 2 is defined, the trend value for the user specified KPI will be calculated to be used for the next drilldown
    if (level2 != '') & (l1_items != ''):
      global kpiTrend1
      kpiTrend1 = result1[f'{kpi1}_Trend'].loc[result1[level1] == l1_items]
      kpiTrend1 = kpiTrend1.iloc[0]
      
    result1 = result1[['timestamp', f'{level1}', f'{L1KPI}_Trend', f'{kpi1}_Trend_Decomp', f'Agg_{L1KPI}_Trend_Decomp', f'{kpi1}_Business_Mix', f'{kpi1}_Impact_%{kpi1}', f'{kpi1}_Attribution', f'{kpi1}_Att_Business_Mix', f'{kpi1}_Att_Agg']]
    display(result1)
    return(result1)

In [0]:
def L22_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend1, l1_items, l2_items, ranking_field, ranking_function, bins_cols, bins_size, date_field):
  import glob
  import os
  from datetime import datetime, timedelta
  #import pandasql as ps
  from pyspark.sql import SQLContext
  
  sqlContext = SQLContext(sc)
  import time
  
  #df = pd.read_csv('/dbfs/FileStore/CG/output_chubb/data.csv')
  start = time.time()
  print("file read from dbfs started")
  #Read part train files from DBFS and concatenate into dataframe  
  #file path
  #path = '/dbfs/sg_claims_files/'
  path = file_path
  # check if string "claims" is part of the filepath - to identify claims/loss ratio database
  if 'claims' in path:
    #Read part train files from DBFS and concatenate into dataframe
    all_files = glob.glob(os.path.join(path, "SG_Claims_Training_0.ftr"))     
    df_from_each_file = (pd.read_feather(f) for f in all_files) #all columns
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    # calcualte claims count and policy_cvog count from raw data (since these fields are required to calculate some KPI's it has been hardcoded. Assumption is that these fields will be already available in the data later)
    concatenated_df['CLAIMS_COUNT']= np.where((concatenated_df['ULT_CLAIM_INCURRED'])>0, concatenated_df['CLAIMNUMBER'] + concatenated_df['COVERAGE_GROUP'] , '999')
    concatenated_df['POL_CVG_COUNT']= np.where((concatenated_df['POLICY']).notna(), concatenated_df['POLICY'] + concatenated_df['COVERAGE_GROUP'] , '999')
  #for billing and other database files which are single files in .csv format  
  else:
    #read file
    concatenated_df = pd.read_csv(path)
  
  #Convert date_field to pandas datetime
  concatenated_df[date_field] = pd.to_datetime(concatenated_df[date_field])
  # Date Filter
  concatenated_df = concatenated_df.loc[(concatenated_df[date_field] >= start_date) & (concatenated_df[date_field] <= end_date)]
  
  #Binning columns with user selected column names and bin sizes
  for c in range(len(bins_cols)):
    concatenated_df[bins_cols[c]+'_bins'] = pd.qcut(concatenated_df[bins_cols[c]], q=bins_size[c], precision = 0).astype(str)
  
  #Add month and year columns
  concatenated_df['Accd_month'] = pd.to_datetime(concatenated_df[date_field]).dt.month_name()
  concatenated_df['Accd_year'] = pd.to_datetime(concatenated_df[date_field]).dt.year
  concatenated_df['Accd_quarter'] = pd.to_datetime(concatenated_df[date_field]).dt.quarter
  
  #display(concatenated_df)
  
  end = time.time()
  print("file read from dbfs ended",end - start)
  
  #Filter data on values in the anomaly result case as per the fields selected in Anomaly Detection. 
  if((field1 !="") & (field2!="") & (field3!="")):
      concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2) & ( concatenated_df[field3] == an_item3)]
      
      #display(df)
  
  elif((field1 !="")   & (field2=="") & (field3=="")): 
      concatenated_df = concatenated_df.loc[concatenated_df[field1] == an_item1]
      
      #display(concatenated_df)
  
  elif((field1 !="")   & (field2 !="" ) & (field3=="")):
     concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2)]
    
    #display(concatenated_df)
  
  #l2 data preprocessing------------------------------------------------------------------------
  #Filter on level 1 and level 2 field
  
  
  data = concatenated_df
  #put aggregation columns in a list
  #put aggregation columns in a list
  lst2 = [level1,level2]
  #put columns for year and month in list
  if(period == 'month'):
      segment = ['Accd_year','Accd_month']
      
    #put columns for year and quarter in list
  if(period == 'quarter'):
      segment = ['Accd_year','Accd_quarter']
      
  segment.extend(lst2)
  
  
  if((level1!='') & (level2!='')):
    
    # calculate KPI values for KPIs selected by user
      
    data = concatenated_df.groupby(segment).agg({L1Num: num_function_1 , L1Den: den_function_1, ranking_field: ranking_function })
    #data = concatenated_df.groupby(segment)
    data.reset_index(inplace = True)
    #display(data)
    #if empty data set obtained after aggregation
    if data.empty:
      return('No records found for this selection!')
    #Calculate values for KPI's for each row timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    data[L1KPI] = data[L1Num]/data[L1Den]
    
    kpiTrend = 1
    
    #Create timestamp column
    if(period == 'month'):
      data['Month Name-Year']=pd.to_datetime(data['Accd_month']+data['Accd_year'].astype(str),format='%B%Y')
      data['timestamp'] = data['Month Name-Year'].dt.strftime('%Y-%m')

    if(period == 'quarter'):
      data['Accd_quarter']=data['Accd_quarter'].replace({1:'1', 2: '4', 3: '7', 4: '10'})
      data['quart_year']=pd.to_datetime(data['Accd_year'].astype(str) +data['Accd_quarter'].astype(str),format='%Y%m')
      data['timestamp'] = data['quart_year'].dt.strftime('%Y-%m')
      

    #Create a list of the dates within the rolling window used for causal analysis
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data =  data.sort_values(by='timestamp', ascending=True)
    #Replace invalid values
    data = data.replace('#DIV/0!', np.NaN)
    data = data.replace('#VALUE!', np.NaN)
    #Conversion to float
    data[L1KPI] = data[L1KPI].astype(str).astype(float)
    numeric_columns = data.select_dtypes(include=['number']).columns
    from dateutil.relativedelta import relativedelta
  
    if(period == 'month'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,baseline_window,1))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=baseline_window - 1 - i)
        #past_date = past_date.strftime('%Y-%m-%d')
        timelist.append( datetime(past_date.year, past_date.month, 1).strftime('%Y-%m-%d') )
        #timelist.append(past_date)
      print(timelist)    
      timeArray = np.array(timelist)
      #dateArray = np.array(timelist)
      #print(dateArray, timelist[0].type())
      
    if(period == 'quarter'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,(baseline_window*3),3))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=(baseline_window*3) - 3 - i)
        past_date = past_date.strftime('%Y-%m-%d')
        timelist.append(past_date)
      print(timelist)  
      timeArray = np.array(timelist)
    
    dgroups = []
    for d in timelist:
        locals()[d] = data.loc[data['timestamp'] == d]
        dgroups.append(locals()[d])
    data = pd.concat(dgroups).fillna(0)
    #display(data)
    
    #timeArray is used for the rolling average function. Creates an index with length defined by the 
    #baseline_window variable
    timelist = list(range(0,baseline_window,1))
    timeArray = np.array(timelist)
    len(timeArray)
    data = data.fillna(0)

    group1 = list(data[level1].unique())
    groups1 = []
    for c in group1:
        locals()[c] = data.loc[data[level1] == c]
        groups1.append(locals()[c])
        
    group2 = list(data[level2].unique())
    groups2 = []
    for c in group2:
        locals()[c] = data.loc[data[level2] == c]
        groups2.append(locals()[c])
        
    for g in groups1:
      for s in groups2:
        Array = s[L1KPI].to_numpy()
        Trend = []
        for i in range(0, len(Array), 1):
            if i>(baseline_window-2):
                x = timeArray
                y = Array[timeArray-(baseline_window-1)+i]
                m, b = np.polyfit(x, y, 1)
                Trend.append(m)
            else:
                Trend.append(0)

        Trend = np.asarray(Trend)
        s[f'{L1KPI}_Trend'] = Trend

        #Weighting basis
        s[f'Avg_{L1KPI}'] = s[f'{L1KPI}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L1Num}'] = s[f'{L1Num}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L1Den}'] = s[f'{L1Den}'].rolling(window = baseline_window).mean()

    data = pd.concat(groups2).fillna(0)

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    #Loops over each rolling window and calculates the weights for each occurance
    for s in dates:
        Array = s[f'Avg_{L1Den}'].to_numpy()
        s[f'{L1KPI}_weight_basis'] = Array.sum()
        
    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Weight'] = data[f'Avg_{L1Den}'] / data[f'{L1KPI}_weight_basis']

    #Calculates aggregated weights to ensure it adds up to exactly 1
    data1 = data.loc[data['timestamp'] == an_date]
    data1[f'Agg_Weight'] = data1[f'{L1KPI}_Weight'].sum()

    data[f'{L1KPI}_Slope_%ofAvg'] = data[f'{L1KPI}_Trend'] / data[f'Avg_{L1KPI}']
    data[f'{L1KPI}_Trend_Decomp'] = data[f'{L1KPI}_Weight'] * data[f'{L1KPI}_Trend']

    #Loops over each rolling window to calculate trend decomp and business mix
    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'Agg_{L1KPI}_Trend_Decomp'] = s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'{L1KPI}_Business_Mix'] = kpiTrend - s[f'{L1KPI}_Trend_Decomp'].sum()

    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Attribution'] = data[f'{L1KPI}_Trend_Decomp'] / kpiTrend
    data[f'{L1KPI}_Impact_%{L1KPI}'] = data[f'{L1KPI}_Trend_Decomp'] / data[f'Avg_{L1KPI}']

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'{L1KPI}_Att_Business_Mix'] = s[f'{L1KPI}_Business_Mix'] / kpiTrend
        s[f'{L1KPI}_Att_Agg'] = s[f'{L1KPI}_Attribution'].sum() + s[f'{L1KPI}_Att_Business_Mix']
    data = pd.concat(dates).fillna(0)
    #global kpiTrend1
    #kpiTrend1 = data[f'{kpi1}_Trend']    
    data[f'Agg_Trend_Decomp'] = data[f'{L1KPI}_Trend_Decomp'].sum()
    
    #create result file
    result2 = data.loc[(data['timestamp'] == an_date) & (data[level1] == l1_items)]
    
    #if level 3 is defined, the trend value for the user specified KPI will be calculated to be used for the next drilldown
    if (level3 != '') & (l2_items != ''):
      global kpiTrend2
      kpiTrend2 = result2[f'{kpi1}_Trend'].loc[result2[level2] == l2_items]
      kpiTrend2 = kpiTrend2.iloc[0]
    #result2 = result2.loc[result2[level1] == l1_items]
    result2 = result2[['timestamp', f'{level2}',f'{level1}', f'{kpi1}_Trend', f'{kpi1}_Trend_Decomp',f'{kpi1}_Business_Mix', f'{kpi1}_Impact_%{kpi1}', f'{kpi1}_Attribution', f'{kpi1}_Att_Business_Mix', f'{kpi1}_Att_Agg']]
    display(result2)
    return(result2)

In [0]:
def L23_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend2, l1_items, l2_items, ranking_field, ranking_function, bins_cols, bins_size, date_field):
  import glob
  import os
  from datetime import datetime, timedelta
  #import pandasql as ps
  from pyspark.sql import SQLContext
  
  sqlContext = SQLContext(sc)
  import time
  
  #df = pd.read_csv('/dbfs/FileStore/CG/output_chubb/data.csv')
  start = time.time()
  print("file read from dbfs started")
  #Read part train files from DBFS and concatenate into dataframe  
  #file path
  #path = '/dbfs/sg_claims_files/'
  path = file_path
  # check if string "claims" is part of the filepath - to identify claims/loss ratio database
  if 'claims' in path:
    #Read part train files from DBFS and concatenate into dataframe
    all_files = glob.glob(os.path.join(path, "SG_Claims_Training_0.ftr"))     
    df_from_each_file = (pd.read_feather(f) for f in all_files) #all columns
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    # calcualte claims count and policy_cvog count from raw data (since these fields are required to calculate some KPI's it has been hardcoded. Assumption is that these fields will be already available in the data later)
    concatenated_df['CLAIMS_COUNT']= np.where((concatenated_df['ULT_CLAIM_INCURRED'])>0, concatenated_df['CLAIMNUMBER'] + concatenated_df['COVERAGE_GROUP'] , '999')
    concatenated_df['POL_CVG_COUNT']= np.where((concatenated_df['POLICY']).notna(), concatenated_df['POLICY'] + concatenated_df['COVERAGE_GROUP'] , '999')
  #for billing and other database files which are single files in .csv format  
  else:
    #read file
    concatenated_df = pd.read_csv(path)
  
  #Convert date_field to pandas datetime
  concatenated_df[date_field] = pd.to_datetime(concatenated_df[date_field])
  # Date Filter
  concatenated_df = concatenated_df.loc[(concatenated_df[date_field] >= start_date) & (concatenated_df[date_field] <= end_date)]
  
  #Binning columns with user selected column names and bin sizes
  for c in range(len(bins_cols)):
    concatenated_df[bins_cols[c]+'_bins'] = pd.qcut(concatenated_df[bins_cols[c]], q=bins_size[c], precision = 0).astype(str)
  
  #Add month and year columns
  concatenated_df['Accd_month'] = pd.to_datetime(concatenated_df[date_field]).dt.month_name()
  concatenated_df['Accd_year'] = pd.to_datetime(concatenated_df[date_field]).dt.year
  concatenated_df['Accd_quarter'] = pd.to_datetime(concatenated_df[date_field]).dt.quarter
  
  #display(concatenated_df)
  
  end = time.time()
  print("file read from dbfs ended",end - start)
  
  #Filter data on values in the anomaly result case as per the fields selected in Anomaly Detection. 
  if((field1 !="") & (field2!="") & (field3!="")):
      concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2) & ( concatenated_df[field3] == an_item3)]
      
      #display(df)
  
  elif((field1 !="")   & (field2=="") & (field3=="")): 
      concatenated_df = concatenated_df.loc[concatenated_df[field1] == an_item1]
      
      #display(concatenated_df)
  
  elif((field1 !="")   & (field2 !="" ) & (field3=="")):
     concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2)]
    
    #display(concatenated_df)
  
  #l2 data preprocessing------------------------------------------------------------------------
  #Filter on level 1 and level 2 field
  
  
  data = concatenated_df
  #put aggregation columns in a list
  #put aggregation columns in a list
  lst3 = [level1,level2,level3]
  #put columns for year and month in list
  if(period == 'month'):
      segment = ['Accd_year','Accd_month']
      
    #put columns for year and quarter in list
  if(period == 'quarter'):
      segment = ['Accd_year','Accd_quarter']
      
  segment.extend(lst3)
  
  
  if((level1!='') & (level2!='') & (level3!='')):
    
    #filter data on level1 values
    concatenated_df = concatenated_df.loc[(concatenated_df[level1] == l1_items) & (concatenated_df[level2] == l2_items)]
      
    data = concatenated_df.groupby(segment).agg({L1Num: num_function_1 , L1Den: den_function_1, ranking_field: ranking_function})
    #data = concatenated_df.groupby(segment)
    data.reset_index(inplace = True)
    #display(data)
    #if empty data set obtained after aggregation
    if data.empty:
      return('No records found for this selection!')
    #Calculate values for KPI's for each row timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    data[L1KPI] = data[L1Num]/data[L1Den]
    
    kpiTrend = 1
    
    #Create timestamp column
    if(period == 'month'):
      data['Month Name-Year']=pd.to_datetime(data['Accd_month']+data['Accd_year'].astype(str),format='%B%Y')
      data['timestamp'] = data['Month Name-Year'].dt.strftime('%Y-%m')

    if(period == 'quarter'):
      data['Accd_quarter']=data['Accd_quarter'].replace({1:'1', 2: '4', 3: '7', 4: '10'})
      data['quart_year']=pd.to_datetime(data['Accd_year'].astype(str) +data['Accd_quarter'].astype(str),format='%Y%m')
      data['timestamp'] = data['quart_year'].dt.strftime('%Y-%m')
      

    #Create a list of the dates within the rolling window used for causal analysis
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data =  data.sort_values(by='timestamp', ascending=True)
    #Replace invalid values
    data = data.replace('#DIV/0!', np.NaN)
    data = data.replace('#VALUE!', np.NaN)
    #Conversion to float
    data[L1KPI] = data[L1KPI].astype(str).astype(float)
    numeric_columns = data.select_dtypes(include=['number']).columns
  
    if(period == 'month'):
      from dateutil.relativedelta import relativedelta
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,baseline_window,1))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=baseline_window - 1 - i)
        #past_date = past_date.strftime('%Y-%m-%d')
        timelist.append( datetime(past_date.year, past_date.month, 1).strftime('%Y-%m-%d') )
        #timelist.append(past_date)
      print(timelist)    
      timeArray = np.array(timelist)
      #dateArray = np.array(timelist)
      #print(dateArray, timelist[0].type())
      
    if(period == 'quarter'):
      from dateutil.relativedelta import relativedelta
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,(baseline_window*3),3))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=(baseline_window*3) - 3 - i)
        past_date = past_date.strftime('%Y-%m-%d')
        timelist.append(past_date)
      print(timelist)  
      timeArray = np.array(timelist)
    
    dgroups = []
    for d in timelist:
        locals()[d] = data.loc[data['timestamp'] == d]
        dgroups.append(locals()[d])
    data = pd.concat(dgroups).fillna(0)
    #display(data)
    
    #timeArray is used for the rolling average function. Creates an index with length defined by the 
    #baseline_window variable
    timelist = list(range(0,baseline_window,1))
    timeArray = np.array(timelist)
    len(timeArray)
    data = data.fillna(0)

    group1 = list(data[level1].unique())
    groups1 = []
    for c in group1:
        locals()[c] = data.loc[data[level1] == c]
        groups1.append(locals()[c])
        
    group2 = list(data[level2].unique())
    groups2 = []
    for c in group2:
        locals()[c] = data.loc[data[level2] == c]
        groups2.append(locals()[c])
        
    group3 = list(data[level3].unique())
    groups3 = []
    for c in group3:
        locals()[c] = data.loc[data[level3] == c]
        groups3.append(locals()[c])
    
    for b in group1:    
      for g in groups2:
        for s in groups3:
          Array = s[L1KPI].to_numpy()
          Trend = []
          for i in range(0, len(Array), 1):
              if i>(baseline_window-2):
                  x = timeArray
                  y = Array[timeArray-(baseline_window-1)+i]
                  m, b = np.polyfit(x, y, 1)
                  Trend.append(m)
              else:
                  Trend.append(0)

          Trend = np.asarray(Trend)
          s[f'{L1KPI}_Trend'] = Trend

          #Weighting basis
          s[f'Avg_{L1KPI}'] = s[f'{L1KPI}'].rolling(window = baseline_window).mean()
          s[f'Avg_{L1Num}'] = s[f'{L1Num}'].rolling(window = baseline_window).mean()
          s[f'Avg_{L1Den}'] = s[f'{L1Den}'].rolling(window = baseline_window).mean()

    data = pd.concat(groups3).fillna(0)

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    #Loops over each rolling window and calculates the weights for each occurance
    for s in dates:
        Array = s[f'Avg_{L1Den}'].to_numpy()
        s[f'{L1KPI}_weight_basis'] = Array.sum()
        
    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Weight'] = data[f'Avg_{L1Den}'] / data[f'{L1KPI}_weight_basis']

    #Calculates aggregated weights to ensure it adds up to exactly 1
    data1 = data.loc[data['timestamp'] == an_date]
    data1[f'Agg_Weight'] = data1[f'{L1KPI}_Weight'].sum()

    data[f'{L1KPI}_Slope_%ofAvg'] = data[f'{L1KPI}_Trend'] / data[f'Avg_{L1KPI}']
    data[f'{L1KPI}_Trend_Decomp'] = data[f'{L1KPI}_Weight'] * data[f'{L1KPI}_Trend']

    #Loops over each rolling window to calculate trend decomp and business mix
    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'Agg_{L1KPI}_Trend_Decomp'] = s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'{L1KPI}_Business_Mix'] = kpiTrend - s[f'{L1KPI}_Trend_Decomp'].sum()

    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Attribution'] = data[f'{L1KPI}_Trend_Decomp'] / kpiTrend
    data[f'{L1KPI}_Impact_%{L1KPI}'] = data[f'{L1KPI}_Trend_Decomp'] / data[f'Avg_{L1KPI}']

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'{L1KPI}_Att_Business_Mix'] = s[f'{L1KPI}_Business_Mix'] / kpiTrend
        s[f'{L1KPI}_Att_Agg'] = s[f'{L1KPI}_Attribution'].sum() + s[f'{L1KPI}_Att_Business_Mix']
    data = pd.concat(dates).fillna(0)
    #global kpiTrend1
    #kpiTrend1 = data[f'{kpi1}_Trend']    
    data[f'Agg_Trend_Decomp'] = data[f'{L1KPI}_Trend_Decomp'].sum()
    
    #create result file
    result3 = data.loc[(data['timestamp'] == an_date) & (data[level1] == l1_items) & (data[level2] == l2_items)]
    result3 = result3[['timestamp', f'{level3}', f'{kpi1}_Trend', f'{kpi1}_Trend_Decomp',f'{kpi1}_Business_Mix', f'{kpi1}_Impact_%{kpi1}', f'{kpi1}_Attribution', f'{kpi1}_Att_Business_Mix', f'{kpi1}_Att_Agg']]
    display(result3)
    return(result3)

In [0]:
def L21_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, ranking_field, ranking_function, bins_cols, bins_size, date_field):
  import glob
  import os
  from datetime import datetime, timedelta
  #import pandasql as ps
  from pyspark.sql import SQLContext
  import pandas as pd
  pd.options.mode.chained_assignment = None
  
  sqlContext = SQLContext(sc)
  import time

  start = time.time()
  print("file read from dbfs started")
  

  #file path
  #path = '/dbfs/sg_claims_files/'
  path = file_path
  # check if string "claims" is part of the filepath - to identify claims/loss ratio database
  if 'claims' in path:
    #Read part train files from DBFS and concatenate into dataframe
    all_files = glob.glob(os.path.join(path, "SG_Claims_Training_0.ftr"))     
    df_from_each_file = (pd.read_feather(f) for f in all_files) #all columns
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    # calcualte claims count and policy_cvog count from raw data (since these fields are required to calculate some KPI's it has been hardcoded. Assumption is that these fields will be already available in the data later)
    concatenated_df['CLAIMS_COUNT']= np.where((concatenated_df['ULT_CLAIM_INCURRED'])>0, concatenated_df['CLAIMNUMBER'] + concatenated_df['COVERAGE_GROUP'] , '999')
    concatenated_df['POL_CVG_COUNT']= np.where((concatenated_df['POLICY']).notna(), concatenated_df['POLICY'] + concatenated_df['COVERAGE_GROUP'] , '999')
  #for billing and other database files which are single files in .csv format  
  else:
    #read file
    concatenated_df = pd.read_csv(path)
  
  #Convert date_field to pandas datetime
  concatenated_df[date_field] = pd.to_datetime(concatenated_df[date_field])
  # Date Filter
  concatenated_df = concatenated_df.loc[(concatenated_df[date_field] >= start_date) & (concatenated_df[date_field] <= end_date)]
  
  #Binning columns with user selected column names and bin sizes
  for c in range(len(bins_cols)):
    concatenated_df[bins_cols[c]+'_bins'] = pd.qcut(concatenated_df[bins_cols[c]], q=bins_size[c], precision = 0).astype(str)
  
  #Add month and year columns
  concatenated_df['Accd_month'] = pd.to_datetime(concatenated_df[date_field]).dt.month_name()
  concatenated_df['Accd_year'] = pd.to_datetime(concatenated_df[date_field]).dt.year
  concatenated_df['Accd_quarter'] = pd.to_datetime(concatenated_df[date_field]).dt.quarter
  
  end = time.time()
  print("file read from dbfs ended",end - start)
  
  #Filter data on values in the anomaly result case as per the fields selected in Anomaly Detection. 
  if((field1 !="") & (field2!="") & (field3!="")):
       concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2) & ( concatenated_df[field3] == an_item3)]
      
      #display(df)
  
  elif((field1 !="")   & (field2=="") & (field3=="")): 
      concatenated_df = concatenated_df.loc[concatenated_df[field1] == an_item1]
      
      #display(concatenated_df)
  
  elif((field1 !="")   & (field2 !="" ) & (field3=="")):
     concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2)]
    
    #display(concatenated_df)
  
  #l1 data preprocessing------------------------------------------------------------------------
  #Filter on level 1 field
  
  #put aggregation columns in a list
  lst1 = [level1]
  #put columns for year and month in list
  if(period == 'month'):
      segment = ['Accd_year','Accd_month']
      
    #put columns for year and quarter in list
  if(period == 'quarter'):
      segment = ['Accd_year','Accd_quarter']
      
  segment.extend(lst1)
  
  if(level1!=''):


    # calculate KPI values for KPIs selected by user
    if((L2KPI1 !="") & (L2KPI2!="") & (L2KPI3!="") & (L2KPI4!="")):
      
      data = concatenated_df.groupby(segment).agg({L1Num: num_function_1 , L1Den: den_function_1, L2Num1:sub_num_function_1, L2Den1: sub_den_function_1, L2Num2:sub_num_function_2, L2Den2:sub_den_function_2, L2Num3:sub_num_function_3, L2Den3:sub_den_function_3, L2Num4:sub_num_function_4, L2Den4:sub_den_function_4, ranking_field: ranking_function})
      #data = concatenated_df.groupby(segment)
      data.reset_index(inplace = True)
      #if empty data set obtained after aggregation
      if data.empty:
        return('No records found for this selection!')
    #Calculate values for KPI's for each row timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    data[L1KPI] = data[L1Num]/data[L1Den]
    data[L2KPI1] = data[L2Num1]/data[L2Den1]
    data[L2KPI2] = data[L2Num2]/data[L2Den2]
    data[L2KPI3] = data[L2Num3]/data[L2Den3]
    data[f'{L2KPI3}'] = data[f'{L2KPI3}'].astype(float)
    data[f'{L2Den3}'] = data[f'{L2Num3}']/data[f'{L2KPI3}']
    
    #Create timestamp column
    if(period == 'month'):
      data['Month Name-Year']=pd.to_datetime(data['Accd_month']+data['Accd_year'].astype(str),format='%B%Y')
      data['timestamp'] = data['Month Name-Year'].dt.strftime('%Y-%m')

    if(period == 'quarter'):
      data['Accd_quarter']=data['Accd_quarter'].replace({1:'1', 2: '4', 3: '7', 4: '10'})
      data['quart_year']=pd.to_datetime(data['Accd_year'].astype(str) +data['Accd_quarter'].astype(str),format='%Y%m')
      data['timestamp'] = data['quart_year'].dt.strftime('%Y-%m')
      


    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data =  data.sort_values(by='timestamp', ascending=True)
    #Replace invalid values
    data = data.replace('#DIV/0!', np.NaN)
    data = data.replace('#VALUE!', np.NaN)
    #Conversion to float
    data[L1KPI] = data[L1KPI].astype(str).astype(float)
    numeric_columns = data.select_dtypes(include=['number']).columns
    from dateutil.relativedelta import relativedelta

    #Create a list of the dates within the rolling window used for causal analysis
    if(period == 'month'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,baseline_window,1))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=baseline_window - 1 - i)
        timelist.append( datetime(past_date.year, past_date.month, 1).strftime('%Y-%m-%d') )
      print(timelist)    
      timeArray = np.array(timelist)
      
    if(period == 'quarter'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,(baseline_window*3),3))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=(baseline_window*3) - 3 - i)
        past_date = past_date.strftime('%Y-%m-%d')
        timelist.append(past_date)
      print(timelist)  
      timeArray = np.array(timelist)
    
    dgroups = []
    for d in timelist:
        locals()[d] = data.loc[data['timestamp'] == d]
        dgroups.append(locals()[d])
    data = pd.concat(dgroups).fillna(0)
    #display(data)
    
    #timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    timelist = list(range(0,baseline_window,1))
    timeArray = np.array(timelist)
    len(timeArray)
    data = data.fillna(0)

    group = list(data[level1].unique())
    groups = []
    for c in group:
        locals()[c] = data.loc[data[level1] == c]
        groups.append(locals()[c])

    for s in groups:
        Array = s[L1KPI].to_numpy()
        Trend = []
        for i in range(0, len(Array), 1):
            if i>(baseline_window-2):
                x = timeArray
                y = Array[timeArray-(baseline_window-1)+i]
                m, b = np.polyfit(x, y, 1)
                Trend.append(m)
            else:
                Trend.append(0)

        Trend = np.asarray(Trend)
        s[f'{L1KPI}_Trend'] = Trend

        #Weighting basis
        s[f'Avg_{L1KPI}'] = s[f'{L1KPI}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L2KPI1}'] = s[f'{L2KPI1}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L2KPI2}'] = s[f'{L2KPI2}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L2KPI3}'] = s[f'{L2KPI3}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L2Num2}'] = s[f'{L2Num2}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L2Den2}'] = s[f'{L2Den2}'].rolling(window = baseline_window).mean()
        s[f'Avg_{L2Den3}'] = s[f'{L2Den3}'].rolling(window = baseline_window).mean()

        #Calculate slope for {L2KPI1} metric
        Array = s[f'{L2KPI1}'].to_numpy()
        Trend = []
        for i in range(0, len(Array), 1):
            if i>(baseline_window-2):
                x = timeArray
                y = Array[timeArray-(baseline_window-1)+i]
                m, b = np.polyfit(x, y, 1)
                Trend.append(m)
            else:
                Trend.append(0)
        Trend = np.asarray(Trend)
        s[f'{L2KPI1}_Trend'] = Trend

        #Calculate slope for {L2KPI2} metric
        Array = s[f'{L2KPI2}'].to_numpy()
        Trend = []
        for i in range(0, len(Array), 1):
            if i>(baseline_window-2):
                x = timeArray
                y = Array[timeArray-(baseline_window-1)+i]
                m, b = np.polyfit(x, y, 1)
                Trend.append(m)
            else:
                Trend.append(0)

        Trend = np.asarray(Trend)
        s[f'{L2KPI2}_Trend'] = Trend

        #Calculate slope for {L2KPI3} metric
        Array = s[f'{L2KPI3}'].to_numpy()
        Trend = []
        for i in range(0, len(Array), 1):
            if i>(baseline_window-2):
                x = timeArray
                y = Array[timeArray-(baseline_window-1)+i]
                m, b = np.polyfit(x, y, 1)
                Trend.append(m)
            else:
                Trend.append(0)

        Trend = np.asarray(Trend)
        s[f'{L2KPI3}_Trend'] = Trend

    data = pd.concat(groups).fillna(0)

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    #Loops over each rolling window and calculates the weights for each occurance
    for s in dates:
        Array = s[f'Avg_{L2KPI3}'].to_numpy()
        s[f'{L1KPI}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Num2}'].to_numpy()
        s[f'{L2KPI1}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Den2}'].to_numpy()
        s[f'{L2KPI2}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Den3}'].to_numpy()
        s[f'{L2KPI3}_weight_basis'] = Array.sum()
    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Weight'] = data[f'Avg_{L2KPI3}'] / data[f'{L1KPI}_weight_basis']
    data[f'{L2KPI1}_Weight'] = data[f'Avg_{L2Num2}'] / data[f'{L2KPI1}_weight_basis']
    data[f'{L2KPI2}_Weight'] = data[f'Avg_{L2Den2}'] / data[f'{L2KPI2}_weight_basis']
    data[f'{L2KPI3}_Weight'] = data[f'Avg_{L2Den3}'] / data[f'{L2KPI3}_weight_basis']

    #Calculates aggregated weights to ensure it adds up to exactly 1
    data1 = data.loc[data['timestamp'] == an_date]
    data1[f'Agg_Weight'] = data1[f'{L1KPI}_Weight'].sum()

    data[f'{L1KPI}_Slope_%ofAvg'] = data[f'{L1KPI}_Trend'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI1}_Slope_%ofAvg'] = data[f'{L2KPI1}_Trend'] / data[f'Avg_{L2KPI1}']
    data[f'{L2KPI2}_Slope_%ofAvg'] = data[f'{L2KPI2}_Trend'] / data[f'Avg_{L2KPI2}']
    data[f'{L2KPI3}_Slope_%ofAvg'] = data[f'{L2KPI3}_Trend'] / data[f'Avg_{L2KPI3}']
    data[f'{L1KPI}_Trend_Decomp'] = data[f'{L1KPI}_Weight'] * data[f'{L1KPI}_Trend']
    data[f'{L2KPI1}_Trend_Decomp'] = data[f'{L2KPI1}_Weight'] * data[f'{L2KPI1}_Trend']
    data[f'{L2KPI2}_Trend_Decomp'] = data[f'{L2KPI2}_Weight'] * data[f'{L2KPI2}_Trend']
    data[f'{L2KPI3}_Trend_Decomp'] = data[f'{L2KPI3}_Weight'] * data[f'{L2KPI3}_Trend']

    #Loops over each rolling window to calculate trend decomp and business mix
    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'Agg_{L1KPI}_Trend_Decomp'] = s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI1}_Trend_Decomp'] = s[f'{L2KPI1}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI2}_Trend_Decomp'] = s[f'{L2KPI2}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI3}_Trend_Decomp'] = s[f'{L2KPI3}_Trend_Decomp'].sum()
        s[f'{L1KPI}_Business_Mix'] = kpiTrend - s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'{L2KPI1}_Business_Mix'] = kpiTrend - s[f'{L2KPI1}_Trend_Decomp'].sum()
        s[f'{L2KPI2}_Business_Mix'] = kpiTrend - s[f'{L2KPI2}_Trend_Decomp'].sum()
        s[f'{L2KPI3}_Business_Mix'] = kpiTrend - s[f'{L2KPI3}_Trend_Decomp'].sum()

    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Attribution'] = data[f'{L1KPI}_Trend_Decomp'] / kpiTrend
    data[f'{L2KPI1}_Attribution'] = data[f'{L2KPI1}_Trend_Decomp'] / kpiTrend
    data[f'{L2KPI2}_Attribution'] = data[f'{L2KPI2}_Trend_Decomp'] / kpiTrend
    data[f'{L2KPI3}_Attribution'] = data[f'{L2KPI3}_Trend_Decomp'] / kpiTrend
    data[f'{L1KPI}_Impact_%{L1KPI}'] = data[f'{L1KPI}_Trend_Decomp'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI1}_Impact_%{L2KPI1}'] = data[f'{L2KPI1}_Trend_Decomp'] / data[f'Avg_{L2KPI1}']
    data[f'{L2KPI2}_Impact_%{L2KPI2}'] = data[f'{L2KPI2}_Trend_Decomp'] / data[f'Avg_{L2KPI2}']
    data[f'{L2KPI3}_Impact_%{L2KPI3}'] = data[f'{L2KPI3}_Trend_Decomp'] / data[f'Avg_{L2KPI3}']

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'{L1KPI}_Att_Business_Mix'] = s[f'{L1KPI}_Business_Mix'] / kpiTrend
        s[f'{L1KPI}_Att_Agg'] = s[f'{L1KPI}_Attribution'].sum() + s[f'{L1KPI}_Att_Business_Mix']
        s[f'{L2KPI1}_Att_Business_Mix'] = s[f'{L2KPI1}_Business_Mix'] / kpiTrend
        s[f'{L2KPI1}_Att_Agg'] = s[f'{L2KPI1}_Attribution'].sum() + s[f'{L2KPI1}_Att_Business_Mix']
        s[f'{L2KPI2}_Att_Business_Mix'] = s[f'{L2KPI2}_Business_Mix'] / kpiTrend
        s[f'{L2KPI2}_Att_Agg'] = s[f'{L2KPI2}_Attribution'].sum() + s[f'{L2KPI2}_Att_Business_Mix']
        s[f'{L2KPI3}_Att_Business_Mix'] = s[f'{L2KPI3}_Business_Mix'] / kpiTrend
        s[f'{L2KPI3}_Att_Agg'] = s[f'{L2KPI3}_Attribution'].sum() + s[f'{L2KPI3}_Att_Business_Mix']
    data = pd.concat(dates).fillna(0)
    #global kpiTrend1
    #kpiTrend1 = data[f'{kpi1}_Trend']    
    data[f'Agg_Trend_Decomp'] = data[f'{L1KPI}_Trend_Decomp'].sum()
    result1 = data.loc[data['timestamp'] == an_date]
    
    #if level 2 is defined, the trend value for the user specified KPI will be calculated to be used for the next drilldown
    if level2 != '':
      global kpiTrend1
      kpiTrend1 = result1[f'{kpi1}_Trend'].loc[result1[level1] == l1_items]
      kpiTrend1 = kpiTrend1.iloc[0]
      
    result1 = result1[['timestamp', f'{level1}', f'{L1KPI}_Trend', f'{kpi1}_Trend_Decomp', f'Agg_{L1KPI}_Trend_Decomp', f'{kpi1}_Business_Mix', f'{kpi1}_Impact_%{kpi1}', f'{kpi1}_Attribution', f'{kpi1}_Att_Business_Mix', f'{kpi1}_Att_Agg']]
    display(result1)
    return(result1)

In [0]:
def L22_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, l2_items, kpiTrend1, ranking_field, ranking_function, bins_cols, bins_size, date_field):
  import glob
  import os
  from datetime import datetime, timedelta
  #import pandasql as ps
  from pyspark.sql import SQLContext
  
  sqlContext = SQLContext(sc)
  import time
  
  #df = pd.read_csv('/dbfs/FileStore/CG/output_chubb/data.csv')
  start = time.time()
  print("file read from dbfs started")
  #Read part train files from DBFS and concatenate into dataframe  
  #file path
  #path = '/dbfs/sg_claims_files/'
  path = file_path
  # check if string "claims" is part of the filepath - to identify claims/loss ratio database
  if 'claims' in path:
    #Read part train files from DBFS and concatenate into dataframe
    all_files = glob.glob(os.path.join(path, "SG_Claims_Training_0.ftr"))     
    df_from_each_file = (pd.read_feather(f) for f in all_files) #all columns
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    # calcualte claims count and policy_cvog count from raw data (since these fields are required to calculate some KPI's it has been hardcoded. Assumption is that these fields will be already available in the data later)
    concatenated_df['CLAIMS_COUNT']= np.where((concatenated_df['ULT_CLAIM_INCURRED'])>0, concatenated_df['CLAIMNUMBER'] + concatenated_df['COVERAGE_GROUP'] , '999')
    concatenated_df['POL_CVG_COUNT']= np.where((concatenated_df['POLICY']).notna(), concatenated_df['POLICY'] + concatenated_df['COVERAGE_GROUP'] , '999')
  #for billing and other database files which are single files in .csv format  
  else:
    #read file
    concatenated_df = pd.read_csv(path)
  
  #Convert date_field to pandas datetime
  concatenated_df[date_field] = pd.to_datetime(concatenated_df[date_field])
  # Date Filter
  concatenated_df = concatenated_df.loc[(concatenated_df[date_field] >= start_date) & (concatenated_df[date_field] <= end_date)]
  
  #Binning columns with user selected column names and bin sizes
  for c in range(len(bins_cols)):
    concatenated_df[bins_cols[c]+'_bins'] = pd.qcut(concatenated_df[bins_cols[c]], q=bins_size[c], precision = 0).astype(str)
  
  #Add month and year columns
  concatenated_df['Accd_month'] = pd.to_datetime(concatenated_df[date_field]).dt.month_name()
  concatenated_df['Accd_year'] = pd.to_datetime(concatenated_df[date_field]).dt.year
  concatenated_df['Accd_quarter'] = pd.to_datetime(concatenated_df[date_field]).dt.quarter
  
  #display(concatenated_df)
  
  end = time.time()
  print("file read from dbfs ended",end - start)
  
  #Filter data on values in the anomaly result case as per the fields selected in Anomaly Detection. 
  if((field1 !="") & (field2!="") & (field3!="")):
      concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2) & ( concatenated_df[field3] == an_item3)]
      
      #display(df)
  
  elif((field1 !="")   & (field2=="") & (field3=="")): 
      concatenated_df = concatenated_df.loc[concatenated_df[field1] == an_item1]
      
      #display(concatenated_df)
  
  elif((field1 !="")   & (field2 !="" ) & (field3=="")):
     concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2)]
    
    #display(concatenated_df)
  
  #l2 data preprocessing------------------------------------------------------------------------
  #Filter on level 1 and level 2 field
  
  
  data = concatenated_df
  #put aggregation columns in a list
  #put aggregation columns in a list
  lst2 = [level1,level2]
  #put columns for year and month in list
  if(period == 'month'):
      segment = ['Accd_year','Accd_month']
      
    #put columns for year and quarter in list
  if(period == 'quarter'):
      segment = ['Accd_year','Accd_quarter']
      
  segment.extend(lst2)
  
  
  if((level1!='') & (level2!='')):
    
    #filter data on level1 values
    concatenated_df = concatenated_df.loc[concatenated_df[level1] == l1_items]
    

    # calculate KPI values for KPIs selected by user
    if((L1KPI !="") & (L2KPI1 !="") & (L2KPI2 !="") & (L2KPI3 !="")):
      
      data = concatenated_df.groupby(segment).agg({L1Num: num_function_1 , L1Den: den_function_1, L2Num1:sub_num_function_1, L2Den1: sub_den_function_1, L2Num2:sub_num_function_2, L2Den2:sub_den_function_2, L2Num3:sub_num_function_3, L2Den3:sub_den_function_3, L2Num4:sub_num_function_4, L2Den4:sub_den_function_4, ranking_field: ranking_function})
      #data = concatenated_df.groupby(segment)
      data.reset_index(inplace = True)
      #display(data)
      #if empty data set obtained after aggregation
      if data.empty:
        return('No records found for this selection!')
    #Calculate values for KPI's for each row timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    data[L1KPI] = data[L1Num]/data[L1Den]
    data[L2KPI1] = data[L2Num1]/data[L2Den1]
    data[L2KPI2] = data[L2Num2]/data[L2Den2]
    data[L2KPI3] = data[L2Num3]/data[L2Den3]
    data[f'{L2KPI3}'] = data[f'{L2KPI3}'].astype(float)
    data[f'{L2Den3}'] = data[f'{L2Num3}']/data[f'{L2KPI3}']
    
    #Create timestamp column
    if(period == 'month'):
      data['Month Name-Year']=pd.to_datetime(data['Accd_month']+data['Accd_year'].astype(str),format='%B%Y')
      data['timestamp'] = data['Month Name-Year'].dt.strftime('%Y-%m')

    if(period == 'quarter'):
      data['Accd_quarter']=data['Accd_quarter'].replace({1:'1', 2: '4', 3: '7', 4: '10'})
      data['quart_year']=pd.to_datetime(data['Accd_year'].astype(str) +data['Accd_quarter'].astype(str),format='%Y%m')
      data['timestamp'] = data['quart_year'].dt.strftime('%Y-%m')
      


    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data =  data.sort_values(by='timestamp', ascending=True)
    #Replace invalid values
    data = data.replace('#DIV/0!', np.NaN)
    data = data.replace('#VALUE!', np.NaN)
    #Conversion to float
    data[L1KPI] = data[L1KPI].astype(str).astype(float)
    numeric_columns = data.select_dtypes(include=['number']).columns
    from dateutil.relativedelta import relativedelta
  
    #Create a list of the dates within the rolling window used for causal analysis
    if(period == 'month'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,baseline_window,1))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=baseline_window - 1 - i)
        timelist.append( datetime(past_date.year, past_date.month, 1).strftime('%Y-%m-%d') )
      print(timelist)    
      timeArray = np.array(timelist)
      
    if(period == 'quarter'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,(baseline_window*3),3))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=(baseline_window*3) - 3 - i)
        past_date = past_date.strftime('%Y-%m-%d')
        timelist.append(past_date)
      print(timelist)  
      timeArray = np.array(timelist)
    
    dgroups = []
    for d in timelist:
        locals()[d] = data.loc[data['timestamp'] == d]
        dgroups.append(locals()[d])
    data = pd.concat(dgroups).fillna(0)
    #display(data)
    
    #timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    timelist = list(range(0,baseline_window,1))
    timeArray = np.array(timelist)
    len(timeArray)
    data = data.fillna(0)


    group1 = list(data[level1].unique())
    groups1 = []
    for c in group1:
        locals()[c] = data.loc[data[level1] == c]
        groups1.append(locals()[c])
        
    group2 = list(data[level2].unique())
    groups2 = []
    for c in group2:
        locals()[c] = data.loc[data[level2] == c]
        groups2.append(locals()[c])
        
    for g in groups1:
      for s in groups2:
          Array = s[L1KPI].to_numpy()
          Trend = []
          for i in range(0, len(Array), 1):
              if i>(baseline_window-2):
                  x = timeArray
                  y = Array[timeArray-(baseline_window-1)+i]
                  m, b = np.polyfit(x, y, 1)
                  Trend.append(m)
              else:
                  Trend.append(0)

          Trend = np.asarray(Trend)
          s[f'{L1KPI}_Trend'] = Trend

          #Weighting basis
          s[f'Avg_{L1KPI}'] = s[f'{L1KPI}'].rolling(window = baseline_window).mean()
          s[f'Avg_{L2KPI1}'] = s[f'{L2KPI1}'].rolling(window = baseline_window).mean()
          s[f'Avg_{L2KPI2}'] = s[f'{L2KPI2}'].rolling(window = baseline_window).mean()
          s[f'Avg_{L2KPI3}'] = s[f'{L2KPI3}'].rolling(window = baseline_window).mean()
          s[f'Avg_{L2Num2}'] = s[f'{L2Num2}'].rolling(window = baseline_window).mean()
          s[f'Avg_{L2Den2}'] = s[f'{L2Den2}'].rolling(window = baseline_window).mean()
          s[f'Avg_{L2Den3}'] = s[f'{L2Den3}'].rolling(window = baseline_window).mean()

          #Calculate slope for {L2KPI1} metric
          Array = s[f'{L2KPI1}'].to_numpy()
          Trend = []
          for i in range(0, len(Array), 1):
              if i>(baseline_window-2):
                  x = timeArray
                  y = Array[timeArray-(baseline_window-1)+i]
                  m, b = np.polyfit(x, y, 1)
                  Trend.append(m)
              else:
                  Trend.append(0)
          Trend = np.asarray(Trend)
          s[f'{L2KPI1}_Trend'] = Trend

          #Calculate slope for {L2KPI2} metric
          Array = s[f'{L2KPI2}'].to_numpy()
          Trend = []
          for i in range(0, len(Array), 1):
              if i>(baseline_window-2):
                  x = timeArray
                  y = Array[timeArray-(baseline_window-1)+i]
                  m, b = np.polyfit(x, y, 1)
                  Trend.append(m)
              else:
                  Trend.append(0)

          Trend = np.asarray(Trend)
          s[f'{L2KPI2}_Trend'] = Trend

          #Calculate slope for {L2KPI3} metric
          Array = s[f'{L2KPI3}'].to_numpy()
          Trend = []
          for i in range(0, len(Array), 1):
              if i>(baseline_window-2):
                  x = timeArray
                  y = Array[timeArray-(baseline_window-1)+i]
                  m, b = np.polyfit(x, y, 1)
                  Trend.append(m)
              else:
                  Trend.append(0)

          Trend = np.asarray(Trend)
          s[f'{L2KPI3}_Trend'] = Trend
        #data = pd.concat(groups2).fillna(0)
      data = pd.concat(groups2).fillna(0)

      Array = s[L1KPI].to_numpy()
      Trend = []
      for i in range(0, len(Array), 1):
          if i>(baseline_window-2):
              x = timeArray
              y = Array[timeArray-(baseline_window-1)+i]
              m, b = np.polyfit(x, y, 1)
              Trend.append(m)
          else:
              Trend.append(0)

      Trend = np.asarray(Trend)
      s[f'{L1KPI}_Trend'] = Trend
      
    #display(data)

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    #Loops over each rolling window and calculates the weights for each occurance
    for s in dates:
        Array = s[f'Avg_{L2KPI3}'].to_numpy()
        s[f'{L1KPI}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Num2}'].to_numpy()
        s[f'{L2KPI1}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Den2}'].to_numpy()
        s[f'{L2KPI2}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Den3}'].to_numpy()
        s[f'{L2KPI3}_weight_basis'] = Array.sum()
    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Weight'] = data[f'Avg_{L2KPI3}'] / data[f'{L1KPI}_weight_basis']
    data[f'{L2KPI1}_Weight'] = data[f'Avg_{L2Num2}'] / data[f'{L2KPI1}_weight_basis']
    data[f'{L2KPI2}_Weight'] = data[f'Avg_{L2Den2}'] / data[f'{L2KPI2}_weight_basis']
    data[f'{L2KPI3}_Weight'] = data[f'Avg_{L2Den3}'] / data[f'{L2KPI3}_weight_basis']

    #Calculates aggregated weights to ensure it adds up to exactly 1
    data1 = data.loc[data['timestamp'] == an_date]
    data1[f'Agg_Weight'] = data1[f'{L1KPI}_Weight'].sum()

    data[f'{L1KPI}_Slope_%ofAvg'] = data[f'{L1KPI}_Trend'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI1}_Slope_%ofAvg'] = data[f'{L2KPI1}_Trend'] / data[f'Avg_{L2KPI1}']
    data[f'{L2KPI2}_Slope_%ofAvg'] = data[f'{L2KPI2}_Trend'] / data[f'Avg_{L2KPI2}']
    data[f'{L2KPI3}_Slope_%ofAvg'] = data[f'{L2KPI3}_Trend'] / data[f'Avg_{L2KPI3}']
    data[f'{L1KPI}_Trend_Decomp'] = data[f'{L1KPI}_Weight'] * data[f'{L1KPI}_Trend']
    data[f'{L2KPI1}_Trend_Decomp'] = data[f'{L2KPI1}_Weight'] * data[f'{L2KPI1}_Trend']
    data[f'{L2KPI2}_Trend_Decomp'] = data[f'{L2KPI2}_Weight'] * data[f'{L2KPI2}_Trend']
    data[f'{L2KPI3}_Trend_Decomp'] = data[f'{L2KPI3}_Weight'] * data[f'{L2KPI3}_Trend']

    #Loops over each rolling window to calculate trend decomp and business mix
    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'Agg_{L1KPI}_Trend_Decomp'] = s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI1}_Trend_Decomp'] = s[f'{L2KPI1}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI2}_Trend_Decomp'] = s[f'{L2KPI2}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI3}_Trend_Decomp'] = s[f'{L2KPI3}_Trend_Decomp'].sum()
        s[f'{L1KPI}_Business_Mix'] = kpiTrend1 - s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'{L2KPI1}_Business_Mix'] = kpiTrend1 - s[f'{L2KPI1}_Trend_Decomp'].sum()
        s[f'{L2KPI2}_Business_Mix'] = kpiTrend1 - s[f'{L2KPI2}_Trend_Decomp'].sum()
        s[f'{L2KPI3}_Business_Mix'] = kpiTrend1 - s[f'{L2KPI3}_Trend_Decomp'].sum()

    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Attribution'] = data[f'{L1KPI}_Trend_Decomp'] / kpiTrend1
    data[f'{L2KPI1}_Attribution'] = data[f'{L2KPI1}_Trend_Decomp'] / kpiTrend1
    data[f'{L2KPI2}_Attribution'] = data[f'{L2KPI2}_Trend_Decomp'] / kpiTrend1
    data[f'{L2KPI3}_Attribution'] = data[f'{L2KPI3}_Trend_Decomp'] / kpiTrend1
    data[f'{L1KPI}_Impact_%{L1KPI}'] = data[f'{L1KPI}_Trend_Decomp'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI1}_Impact_%{L2KPI1}'] = data[f'{L2KPI1}_Trend_Decomp'] / data[f'Avg_{L2KPI1}']
    data[f'{L2KPI2}_Impact_%{L2KPI2}'] = data[f'{L2KPI2}_Trend_Decomp'] / data[f'Avg_{L2KPI2}']
    data[f'{L2KPI3}_Impact_%{L2KPI3}'] = data[f'{L2KPI3}_Trend_Decomp'] / data[f'Avg_{L2KPI3}']

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'{L1KPI}_Att_Business_Mix'] = s[f'{L1KPI}_Business_Mix'] / kpiTrend1
        s[f'{L1KPI}_Att_Agg'] = s[f'{L1KPI}_Attribution'].sum() + s[f'{L1KPI}_Att_Business_Mix']
        s[f'{L2KPI1}_Att_Business_Mix'] = s[f'{L2KPI1}_Business_Mix'] / kpiTrend1
        s[f'{L2KPI1}_Att_Agg'] = s[f'{L2KPI1}_Attribution'].sum() + s[f'{L2KPI1}_Att_Business_Mix']
        s[f'{L2KPI2}_Att_Business_Mix'] = s[f'{L2KPI2}_Business_Mix'] / kpiTrend1
        s[f'{L2KPI2}_Att_Agg'] = s[f'{L2KPI2}_Attribution'].sum() + s[f'{L2KPI2}_Att_Business_Mix']
        s[f'{L2KPI3}_Att_Business_Mix'] = s[f'{L2KPI3}_Business_Mix'] / kpiTrend1
        s[f'{L2KPI3}_Att_Agg'] = s[f'{L2KPI3}_Attribution'].sum() + s[f'{L2KPI3}_Att_Business_Mix']
    data = pd.concat(dates).fillna(0)
    #global kpiTrend2
    #kpiTrend2 = data[f'{kpi1}_Trend']
    #data1 = data.loc[data['timestamp'] == '2019-04']
    data[f'Agg_Trend_Decomp'] = data[f'{L1KPI}_Trend_Decomp'].sum()
    #data2 = data.loc[data['timestamp'] == an_date]
    #display(data2)
    result2 = data.loc[(data['timestamp'] == an_date) & (data[level1] == l1_items)]
    
    #if level 3 is defined, the trend value for the user specified KPI will be calculated to be used for the next drilldown
    if level3 != '':
      global kpiTrend2
      kpiTrend2 = result2[f'{kpi1}_Trend'].loc[result2[level2] == l2_items]
      kpiTrend2 = kpiTrend2.iloc[0]
    result2 = result2[['timestamp', f'{level2}',f'{level1}', f'{kpi1}_Trend', f'{kpi1}_Trend_Decomp',f'{kpi1}_Business_Mix', f'{kpi1}_Impact_%{kpi1}', f'{kpi1}_Attribution', f'{kpi1}_Att_Business_Mix', f'{kpi1}_Att_Agg']]
    display(result2)
    return(result2)

In [0]:
def L23_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, l2_items, kpiTrend2, ranking_field, ranking_function, bins_cols, bins_size, date_field):
  import glob
  import os
  from datetime import datetime, timedelta
  #import pandasql as ps
  from pyspark.sql import SQLContext
  
  sqlContext = SQLContext(sc)
  import time
  
  #df = pd.read_csv('/dbfs/FileStore/CG/output_chubb/data.csv')
  start = time.time()
  print("file read from dbfs started")
  #Read part train files from DBFS and concatenate into dataframe  
  #file path
  #path = '/dbfs/sg_claims_files/'
  path = file_path
  # check if string "claims" is part of the filepath - to identify claims/loss ratio database
  if 'claims' in path:
    #Read part train files from DBFS and concatenate into dataframe
    all_files = glob.glob(os.path.join(path, "SG_Claims_Training_0.ftr"))     
    df_from_each_file = (pd.read_feather(f) for f in all_files) #all columns
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    # calcualte claims count and policy_cvog count from raw data (since these fields are required to calculate some KPI's it has been hardcoded. Assumption is that these fields will be already available in the data later)
    concatenated_df['CLAIMS_COUNT']= np.where((concatenated_df['ULT_CLAIM_INCURRED'])>0, concatenated_df['CLAIMNUMBER'] + concatenated_df['COVERAGE_GROUP'] , '999')
    concatenated_df['POL_CVG_COUNT']= np.where((concatenated_df['POLICY']).notna(), concatenated_df['POLICY'] + concatenated_df['COVERAGE_GROUP'] , '999')
  #for billing and other database files which are single files in .csv format  
  else:
    #read file
    concatenated_df = pd.read_csv(path)
  
  #Convert date_field to pandas datetime
  concatenated_df[date_field] = pd.to_datetime(concatenated_df[date_field])
  # Date Filter
  concatenated_df = concatenated_df.loc[(concatenated_df[date_field] >= start_date) & (concatenated_df[date_field] <= end_date)]
  
  #Binning columns with user selected column names and bin sizes
  for c in range(len(bins_cols)):
    concatenated_df[bins_cols[c]+'_bins'] = pd.qcut(concatenated_df[bins_cols[c]], q=bins_size[c], precision = 0).astype(str)
  
  #Add month and year columns
  concatenated_df['Accd_month'] = pd.to_datetime(concatenated_df[date_field]).dt.month_name()
  concatenated_df['Accd_year'] = pd.to_datetime(concatenated_df[date_field]).dt.year
  concatenated_df['Accd_quarter'] = pd.to_datetime(concatenated_df[date_field]).dt.quarter
  
  #display(concatenated_df)
  
  end = time.time()
  print("file read from dbfs ended",end - start)
  
  #Filter data on values in the anomaly result case as per the fields selected in Anomaly Detection. 
  if((field1 !="") & (field2!="") & (field3!="")):
      concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2) & ( concatenated_df[field3] == an_item3)]
      
      #display(df)
  
  elif((field1 !="")   & (field2=="") & (field3=="")): 
      concatenated_df = concatenated_df.loc[concatenated_df[field1] == an_item1]
      
      #display(concatenated_df)
  
  elif((field1 !="")   & (field2 !="" ) & (field3=="")):
     concatenated_df =  concatenated_df[( concatenated_df[field1] == an_item1) & ( concatenated_df[field2] == an_item2)]
    
    #display(concatenated_df)
  
  #l2 data preprocessing------------------------------------------------------------------------
  #Filter on level 1 and level 2 field
  
  
  data = concatenated_df
  #put aggregation columns in a list
  #put aggregation columns in a list
  lst3 = [level1,level2,level3]
  #put columns for year and month in list
  if(period == 'month'):
      segment = ['Accd_year','Accd_month']
      
    #put columns for year and quarter in list
  if(period == 'quarter'):
      segment = ['Accd_year','Accd_quarter']
      
  segment.extend(lst3)
  
  
  if((level1!='') & (level2!='') & (level3!='')):
    
    #filter data on level1 values
    concatenated_df = concatenated_df.loc[(concatenated_df[level1] == l1_items) & (concatenated_df[level2] == l2_items)]
    

    # calculate KPI values for KPIs selected by user
    if((L1KPI !="") & (L2KPI1 !="") & (L2KPI2 !="") & (L2KPI3 !="")):
      
      data = concatenated_df.groupby(segment).agg({L1Num: num_function_1 , L1Den: den_function_1, L2Num1:sub_num_function_1, L2Den1: sub_den_function_1, L2Num2:sub_num_function_2, L2Den2:sub_den_function_2, L2Num3:sub_num_function_3, L2Den3:sub_den_function_3, L2Num4:sub_num_function_4, L2Den4:sub_den_function_4, ranking_field: ranking_function})
      #data = concatenated_df.groupby(segment)
      data.reset_index(inplace = True)
      #display(data)
      #if empty data set obtained after aggregation
      if data.empty:
        return('No records found for this selection!')
    #Calculate values for KPI's for each row timeArray is used for the rolling average function. Creates an index with length defined by the baseline_window variable
    data[L1KPI] = data[L1Num]/data[L1Den]
    data[L2KPI1] = data[L2Num1]/data[L2Den1]
    data[L2KPI2] = data[L2Num2]/data[L2Den2]
    data[L2KPI3] = data[L2Num3]/data[L2Den3]
    data[f'{L2KPI3}'] = data[f'{L2KPI3}'].astype(float)
    data[f'{L2Den3}'] = data[f'{L2Num3}']/data[f'{L2KPI3}']
    
    #Create timestamp column
    if(period == 'month'):
      data['Month Name-Year']=pd.to_datetime(data['Accd_month']+data['Accd_year'].astype(str),format='%B%Y')
      data['timestamp'] = data['Month Name-Year'].dt.strftime('%Y-%m')

    if(period == 'quarter'):
      data['Accd_quarter']=data['Accd_quarter'].replace({1:'1', 2: '4', 3: '7', 4: '10'})
      data['quart_year']=pd.to_datetime(data['Accd_year'].astype(str) +data['Accd_quarter'].astype(str),format='%Y%m')
      data['timestamp'] = data['quart_year'].dt.strftime('%Y-%m')
      


    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data =  data.sort_values(by='timestamp', ascending=True)
    #Replace invalid values
    data = data.replace('#DIV/0!', np.NaN)
    data = data.replace('#VALUE!', np.NaN)
    #Conversion to float
    data[L1KPI] = data[L1KPI].astype(str).astype(float)
    numeric_columns = data.select_dtypes(include=['number']).columns
    from dateutil.relativedelta import relativedelta
    
    #Create a list of the dates within the rolling window used for causal analysis
    if(period == 'month'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,baseline_window,1))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=baseline_window - 1 - i)
        timelist.append( datetime(past_date.year, past_date.month, 1).strftime('%Y-%m-%d') )
      print(timelist)    
      timeArray = np.array(timelist)
      
    if(period == 'quarter'):
      an_date = pd.to_datetime(an_date)
      numlist = list(range(0,(baseline_window*3),3))
      timelist = []
      for i in numlist:
        past_date = an_date - relativedelta(months=(baseline_window*3) - 3 - i)
        past_date = past_date.strftime('%Y-%m-%d')
        timelist.append(past_date)
      print(timelist)  
      timeArray = np.array(timelist)
    
    dgroups = []
    for d in timelist:
        locals()[d] = data.loc[data['timestamp'] == d]
        dgroups.append(locals()[d])
    data = pd.concat(dgroups).fillna(0)
    #display(data)
    
    #timeArray is used for the rolling average function. Creates an index with length defined by the 
    #baseline_window variable
    timelist = list(range(0,baseline_window,1))
    timeArray = np.array(timelist)
    len(timeArray)
    data = data.fillna(0)

    group1 = list(data[level1].unique())
    groups1 = []
    for c in group1:
        locals()[c] = data.loc[data[level1] == c]
        groups1.append(locals()[c])
        
    group2 = list(data[level2].unique())
    groups2 = []
    for c in group2:
        locals()[c] = data.loc[data[level2] == c]
        groups2.append(locals()[c])
        
    group3 = list(data[level3].unique())
    groups3 = []
    for c in group3:
        locals()[c] = data.loc[data[level3] == c]
        groups3.append(locals()[c])
    
    for b in group1:    
      for g in groups2:
        for s in groups3:
            Array = s[L1KPI].to_numpy()
            Trend = []
            for i in range(0, len(Array), 1):
                if i>(baseline_window-2):
                    x = timeArray
                    y = Array[timeArray-(baseline_window-1)+i]
                    m, b = np.polyfit(x, y, 1)
                    Trend.append(m)
                else:
                    Trend.append(0)

            Trend = np.asarray(Trend)
            s[f'{L1KPI}_Trend'] = Trend

            #Weighting basis
            s[f'Avg_{L1KPI}'] = s[f'{L1KPI}'].rolling(window = baseline_window).mean()
            s[f'Avg_{L2KPI1}'] = s[f'{L2KPI1}'].rolling(window = baseline_window).mean()
            s[f'Avg_{L2KPI2}'] = s[f'{L2KPI2}'].rolling(window = baseline_window).mean()
            s[f'Avg_{L2KPI3}'] = s[f'{L2KPI3}'].rolling(window = baseline_window).mean()
            s[f'Avg_{L2Num2}'] = s[f'{L2Num2}'].rolling(window = baseline_window).mean()
            s[f'Avg_{L2Den2}'] = s[f'{L2Den2}'].rolling(window = baseline_window).mean()
            s[f'Avg_{L2Den3}'] = s[f'{L2Den3}'].rolling(window = baseline_window).mean()

            #Calculate slope for {L2KPI1} metric
            Array = s[f'{L2KPI1}'].to_numpy()
            Trend = []
            for i in range(0, len(Array), 1):
                if i>(baseline_window-2):
                    x = timeArray
                    y = Array[timeArray-(baseline_window-1)+i]
                    m, b = np.polyfit(x, y, 1)
                    Trend.append(m)
                else:
                    Trend.append(0)
            Trend = np.asarray(Trend)
            s[f'{L2KPI1}_Trend'] = Trend

            #Calculate slope for {L2KPI2} metric
            Array = s[f'{L2KPI2}'].to_numpy()
            Trend = []
            for i in range(0, len(Array), 1):
                if i>(baseline_window-2):
                    x = timeArray
                    y = Array[timeArray-(baseline_window-1)+i]
                    m, b = np.polyfit(x, y, 1)
                    Trend.append(m)
                else:
                    Trend.append(0)

            Trend = np.asarray(Trend)
            s[f'{L2KPI2}_Trend'] = Trend

            #Calculate slope for {L2KPI3} metric
            Array = s[f'{L2KPI3}'].to_numpy()
            Trend = []
            for i in range(0, len(Array), 1):
                if i>(baseline_window-2):
                    x = timeArray
                    y = Array[timeArray-(baseline_window-1)+i]
                    m, b = np.polyfit(x, y, 1)
                    Trend.append(m)
                else:
                    Trend.append(0)

            Trend = np.asarray(Trend)
            s[f'{L2KPI3}_Trend'] = Trend
        data = pd.concat(groups3).fillna(0)

        Array = s[L1KPI].to_numpy()
        Trend = []
        for i in range(0, len(Array), 1):
            if i>(baseline_window-2):
                x = timeArray
                y = Array[timeArray-(baseline_window-1)+i]
                m, b = np.polyfit(x, y, 1)
                Trend.append(m)
            else:
                Trend.append(0)

        Trend = np.asarray(Trend)
        s[f'{L1KPI}_Trend'] = Trend

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    #Loops over each rolling window and calculates the weights for each occurance
    for s in dates:
        Array = s[f'Avg_{L2KPI3}'].to_numpy()
        s[f'{L1KPI}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Num2}'].to_numpy()
        s[f'{L2KPI1}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Den2}'].to_numpy()
        s[f'{L2KPI2}_weight_basis'] = Array.sum()

        Array = s[f'Avg_{L2Den3}'].to_numpy()
        s[f'{L2KPI3}_weight_basis'] = Array.sum()
    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Weight'] = data[f'Avg_{L2KPI3}'] / data[f'{L1KPI}_weight_basis']
    data[f'{L2KPI1}_Weight'] = data[f'Avg_{L2Num2}'] / data[f'{L2KPI1}_weight_basis']
    data[f'{L2KPI2}_Weight'] = data[f'Avg_{L2Den2}'] / data[f'{L2KPI2}_weight_basis']
    data[f'{L2KPI3}_Weight'] = data[f'Avg_{L2Den3}'] / data[f'{L2KPI3}_weight_basis']

    #Calculates aggregated weights to ensure it adds up to exactly 1
    data1 = data.loc[data['timestamp'] == an_date]
    data1[f'Agg_Weight'] = data1[f'{L1KPI}_Weight'].sum()

    data[f'{L1KPI}_Slope_%ofAvg'] = data[f'{L1KPI}_Trend'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI1}_Slope_%ofAvg'] = data[f'{L2KPI1}_Trend'] / data[f'Avg_{L2KPI1}']
    data[f'{L2KPI2}_Slope_%ofAvg'] = data[f'{L2KPI2}_Trend'] / data[f'Avg_{L2KPI2}']
    data[f'{L2KPI3}_Slope_%ofAvg'] = data[f'{L2KPI3}_Trend'] / data[f'Avg_{L2KPI3}']
    data[f'{L1KPI}_Trend_Decomp'] = data[f'{L1KPI}_Weight'] * data[f'{L1KPI}_Trend']
    data[f'{L2KPI1}_Trend_Decomp'] = data[f'{L2KPI1}_Weight'] * data[f'{L2KPI1}_Trend']
    data[f'{L2KPI2}_Trend_Decomp'] = data[f'{L2KPI2}_Weight'] * data[f'{L2KPI2}_Trend']
    data[f'{L2KPI3}_Trend_Decomp'] = data[f'{L2KPI3}_Weight'] * data[f'{L2KPI3}_Trend']

    #Loops over each rolling window to calculate trend decomp and business mix
    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'Agg_{L1KPI}_Trend_Decomp'] = s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI1}_Trend_Decomp'] = s[f'{L2KPI1}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI2}_Trend_Decomp'] = s[f'{L2KPI2}_Trend_Decomp'].sum()
        s[f'Agg_{L2KPI3}_Trend_Decomp'] = s[f'{L2KPI3}_Trend_Decomp'].sum()
        s[f'{L1KPI}_Business_Mix'] = kpiTrend2 - s[f'{L1KPI}_Trend_Decomp'].sum()
        s[f'{L2KPI1}_Business_Mix'] = kpiTrend2 - s[f'{L2KPI1}_Trend_Decomp'].sum()
        s[f'{L2KPI2}_Business_Mix'] = kpiTrend2 - s[f'{L2KPI2}_Trend_Decomp'].sum()
        s[f'{L2KPI3}_Business_Mix'] = kpiTrend2 - s[f'{L2KPI3}_Trend_Decomp'].sum()

    data = pd.concat(dates).fillna(0)

    data[f'{L1KPI}_Attribution'] = data[f'{L1KPI}_Trend_Decomp'] / kpiTrend2
    data[f'{L2KPI1}_Attribution'] = data[f'{L2KPI1}_Trend_Decomp'] / kpiTrend2
    data[f'{L2KPI2}_Attribution'] = data[f'{L2KPI2}_Trend_Decomp'] / kpiTrend2
    data[f'{L2KPI3}_Attribution'] = data[f'{L2KPI3}_Trend_Decomp'] / kpiTrend2
    data[f'{L1KPI}_Impact_%{L1KPI}'] = data[f'{L1KPI}_Trend_Decomp'] / data[f'Avg_{L1KPI}']
    data[f'{L2KPI1}_Impact_%{L2KPI1}'] = data[f'{L2KPI1}_Trend_Decomp'] / data[f'Avg_{L2KPI1}']
    data[f'{L2KPI2}_Impact_%{L2KPI2}'] = data[f'{L2KPI2}_Trend_Decomp'] / data[f'Avg_{L2KPI2}']
    data[f'{L2KPI3}_Impact_%{L2KPI3}'] = data[f'{L2KPI3}_Trend_Decomp'] / data[f'Avg_{L2KPI3}']

    date = list(data['timestamp'].unique())
    dates = []
    for c in date:
        locals()[c] = data.loc[data['timestamp'] == c]
        dates.append(locals()[c])

    for s in dates:
        s[f'{L1KPI}_Att_Business_Mix'] = s[f'{L1KPI}_Business_Mix'] / kpiTrend2
        s[f'{L1KPI}_Att_Agg'] = s[f'{L1KPI}_Attribution'].sum() + s[f'{L1KPI}_Att_Business_Mix']
        s[f'{L2KPI1}_Att_Business_Mix'] = s[f'{L2KPI1}_Business_Mix'] / kpiTrend2
        s[f'{L2KPI1}_Att_Agg'] = s[f'{L2KPI1}_Attribution'].sum() + s[f'{L2KPI1}_Att_Business_Mix']
        s[f'{L2KPI2}_Att_Business_Mix'] = s[f'{L2KPI2}_Business_Mix'] / kpiTrend2
        s[f'{L2KPI2}_Att_Agg'] = s[f'{L2KPI2}_Attribution'].sum() + s[f'{L2KPI2}_Att_Business_Mix']
        s[f'{L2KPI3}_Att_Business_Mix'] = s[f'{L2KPI3}_Business_Mix'] / kpiTrend2
        s[f'{L2KPI3}_Att_Agg'] = s[f'{L2KPI3}_Attribution'].sum() + s[f'{L2KPI3}_Att_Business_Mix']
    data = pd.concat(dates).fillna(0)
    #data = data.loc[data['timestamp'] == '2019-04']
    data[f'Agg_Trend_Decomp'] = data[f'{L1KPI}_Trend_Decomp'].sum()    
    result3 = data.loc[(data['timestamp'] == an_date) & (data[level1] == l1_items) & (data[level2] == l2_items)]
    result3 = result3[['timestamp', f'{level3}', f'{kpi1}_Trend', f'{kpi1}_Trend_Decomp',f'{kpi1}_Business_Mix', f'{kpi1}_Impact_%{kpi1}', f'{kpi1}_Attribution', f'{kpi1}_Att_Business_Mix', f'{kpi1}_Att_Agg']]
    display(result3)
    return(result3)

In [0]:

from datetime import datetime
from dateutil.relativedelta import relativedelta



#calculate start date of baseline based on window size
def get_baseline_date(period,date, window):
  if(period == 'month'):
    #Difference in months between anomaly date and baseline date defined by window
    temp = datetime.strptime(date, '%Y-%m-%d')- relativedelta(months=window)
    temp = format(temp,'%Y-%m-%d')
    return temp
  
  elif(period == 'quarter'):
    #print("inside date quarter")
    #Difference in quarters between anomaly date and baseline date defined by window
    temp = datetime.strptime(date, '%Y-%m-%d')- relativedelta(months=window*3)
    temp = format(temp,'%Y-%m-%d')
    return temp


def main(file_path, baseline_window, L1KPI, L1Num, L1Den, num_function_1 , den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1 , sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2 , sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3 , sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4 , sub_den_function_4, kpi1, ranking_field, ranking_function, level1, level2,level3, field1, field2, field3,start_date,end_date, period, an_date, an_item1, an_item2, an_item3, l1_items, l2_items, bins_cols, bins_size, date_field):
  import glob
  import os
  #import pandasql as ps
  from pyspark.sql import SQLContext
  pd.set_option('display.max_columns', None)
  #Read Anomaly Detection Result File

  df = pd.read_csv('/dbfs/FileStore/CG/output_chubb/data2.csv')
  result = pd.read_csv('/dbfs/FileStore/CG/output_chubb/anomalies2.csv')
  kpiTrend = df.loc[df[field1] == an_item1]
  kpiTrend = df.loc[df['timestamp'] == an_date]
  kpiTrend = kpiTrend.iloc[0]['Trend']
  print(kpiTrend)
  #call L1 causality function
  
  
  #Call the drill down functions according to the configuration of user defined KPI's and drill down levels
  if((L2KPI1 !="")  & (L2KPI2!="") & (L2KPI3!="") & (L2KPI4 != "")):
    
    temp1= L1_causality(df, result, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, L1KPI, L1Num, L1Den, L2KPI1, L2Num1, L2Den1, L2KPI2, L2Num2, L2Den2, L2KPI3, L2Num3, L2Den3, L2KPI4, L2Num4, L2Den4, an_date, an_item1, an_item2, an_item3)
    
    if((level1 !="")   & (level2=="") & (level3=="")):

      l21 = L21_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, ranking_field, ranking_function, bins_cols, bins_size, date_field)

    elif((level1 !="")   & (level2!="") & (level3=="")):

      l21 = L21_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, ranking_field, ranking_function, bins_cols, bins_size, date_field)

      l22 = L22_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, l2_items, kpiTrend1, ranking_field, ranking_function, bins_cols, bins_size, date_field)

    if((level1 !="")   & ((level2 != '') & (l1_items != '')) & ((level3 != '') & (l2_items != ''))):

      l21 = L21_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, ranking_field, ranking_function, bins_cols, bins_size, date_field)

      l22 = L22_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, l2_items, kpiTrend1, ranking_field, ranking_function, bins_cols, bins_size, date_field)

      l23 = L23_causality(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, L2KPI1, L2Num1, L2Den1, sub_num_function_1, sub_den_function_1, L2KPI2, L2Num2, L2Den2, sub_num_function_2, sub_den_function_2, L2KPI3, L2Num3, L2Den3, sub_num_function_3, sub_den_function_3, L2KPI4, L2Num4, L2Den4, sub_num_function_4, sub_den_function_4, an_date, kpiTrend, l1_items, l2_items, kpiTrend2, ranking_field, ranking_function, bins_cols, bins_size, date_field)
      
      
  elif((L2KPI1 =="")  & (L2KPI2=="") & (L2KPI3=="") & (L2KPI4 == "")):
    
    if((level1 !="")   & (level2=="") & (level3=="")):
      L21_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend, l1_items, ranking_field, ranking_function, bins_cols, bins_size, date_field)
    
    elif((level1 !="")   & (level2!="") & (level3=="")):
    
      L21_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend, l1_items, ranking_field, ranking_function, bins_cols, bins_size, date_field)

      L22_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend, l1_items, l2_items,  ranking_field, ranking_function, bins_cols, bins_size, date_field)
      
    if((level1 !="")   & (level2 !="") & (level3 !="")):
      
      L21_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend, l1_items, ranking_field, ranking_function, bins_cols, bins_size, date_field)

      L22_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend, l1_items, l2_items, ranking_field, ranking_function, bins_cols, bins_size, date_field)
      
      L23_causality_1KPI(df, file_path, level1,level2,level3,kpi1,start_date,end_date,period, baseline_window, field1, field2, field3, an_item1, an_item2, an_item3, L1KPI, L1Num, L1Den, num_function_1, den_function_1, an_date, kpiTrend, l1_items, l2_items, ranking_field, ranking_function, bins_cols, bins_size, date_field)

Preview snapshot and index numbers for anomalies and select one to execute causality analysis.

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/FileStore/CG/output_chubb/data2.csv')
anomaly_indexes = pd.read_csv('/dbfs/FileStore/CG/output_chubb/anomalies2.csv')
anomaly_indexes = anomaly_indexes.loc[anomaly_indexes['anomaly_score'] == 1]
display(anomaly_indexes)
#display(df)

Accd_year,Accd_quarter,SPONSOR_NAME,COVERAGE_GROUP,ULT_CLAIM_INCURRED,ULT_EARNED_PREMIUM,CLAIMS_COUNT,AVG_EARNED_EXP_POLCOVG,POL_CVG_COUNT,ALR,CLAIMS_SEVERITY,CLAIMS_FREQUENCY,AVG_PREMIUM,EXPOSURE,quart_year,timestamp,combo,Trend,Change,anomaly_score,Premium_Score,final_rank
2019,7,MAYBANK,ACCIDENTAL MEDICAL,8212.0,24142.339147712963,13,260.3583569999988,2716,0.3401493098806846,631.6923076923077,0.049931180046585,8.8889319395114,0.0958609561855665,2019-07-01,2019-07-01,"MAYBANK,ACCIDENTAL MEDICAL",0.0307358992315034,0.0289409178178612,1.0,698.7014532048942,6.0
2020,7,MAYBANK,ACCIDENTAL MEDICAL,13520.226899999998,30127.66638550269,20,300.7604019999975,2999,0.4487644919789033,676.0113449999999,0.0664981156661712,10.045904096533071,0.1002868962987654,2020-07-01,2020-07-01,"MAYBANK,ACCIDENTAL MEDICAL",0.0367905735814495,0.0177129558430466,1.0,533.6500243404503,5.0
2017,10,MAYBANK,ACCIDENTAL MEDICAL,1912.61,10492.513062310509,4,119.84257700000032,1429,0.1822833089310286,478.1525,0.0333771193855418,7.342556376704345,0.0838646445066482,2017-10-01,2017-10-01,"MAYBANK,ACCIDENTAL MEDICAL",0.0294787488646375,0.0294787488646375,1.0,309.3061575227809,4.0
2021,7,MAYBANK,ACCIDENTAL MEDICAL,3412.0657,36843.972949311246,6,339.93307099999544,3384,0.0926085171296323,568.6776166666667,0.0176505333310158,10.887698862089612,0.1004530351654832,2021-07-01,2021-07-01,"MAYBANK,ACCIDENTAL MEDICAL",-0.0146994799040206,0.007048682817398,1.0,259.70147905248984,3.0
2019,4,MAYBANK,ACCIDENTAL MEDICAL,7331.69,22056.427704840185,17,248.4780200000006,2593,0.332406049525014,431.2758823529412,0.0684165142655272,8.506142578033238,0.0958264635557271,2019-04-01,2019-04-01,"MAYBANK,ACCIDENTAL MEDICAL",0.0017949814136422,0.0104908242554575,1.0,231.39010675468333,2.0
2021,10,MAYBANK,ACCIDENTAL MEDICAL,5152.836,12328.54459394274,3,114.43814499999984,2088,0.4179597973415038,1717.612,0.0262150352052631,5.904475380240776,0.0548075407088121,2021-10-01,2021-10-01,"MAYBANK,ACCIDENTAL MEDICAL",-0.0026826167379597,0.0120168631660609,1.0,148.15043342208983,1.0


Run Causality Analysis

- I have created a separate Anomaly Detection notebook within the UAT folder named Anomaly_Detection_v3.1_Trend_Causality with new file paths so as not to interfere with testing of other notebooks. The new Anomaly Detection notebook must be used in order for results to be transfered to this one.

- For best results, use either the Sens Slope or Beta Slope trend detection algorithms for anomaly inputs in this notebook. Anomalies detected with point algorithms will still run, but they may or may not have any meaningful slope and results may be less significant. 

- Pass parameters to invoke the main function and fetch results. (In subsequent iterations these parameters will be passed via the user interface)

- Please note that the required libraries will be installed at each run of this notebook (handled in the code) since libraries are not yet available at cluster level.

- baseline_window dictates the number of periods considered in the trend calculations

- The first table output beneath Command 11 where the main function is called will display a list of all the anomalies from the proceeding anomaly detection run. From this, the user must input the date of the anomaly they wish to perform causal analysis on in the variable an_date. They must also provide an input coinciding with the level of aggregation from the anomaly detection run for the an_item1 variable. eg. if anomaly detection was run on the coverage level, the user must provide the specific coverage from the chosen anomaly to perform causal analysis on.

- level1, level2, level3, define the drill down levels. l1_items specifies the specific value of the level the user wishes to drill down into. For example, after drilling down on level1 to SPONSOR_NAME, l1_items should reflect the specific sponsor desired for the next drill down eg. 'AMERICAN EXPRESS'.

- field1,field2, field3 (MUST mirror the aggregation level used in the proceeding anomaly detection run): If anomaly detection was aggregated at the monthly coverage level, the setup here must reflect that for the causality to function properly.

- start_date and end_date : Filter data on selected dates. eg: start_date ="'2018-01-01'", end_date = "'2020-12-31'"

- period = option : 'month' and 'quarter'

- L1KPI, L2KPI1, L2KPI2, L2KPI3, each have their own corresponding numerators (eg.L1Num) and denominators (eg.L1Den) which are used to calculate the KPI's in a user defined manner.

In [0]:
#main(file_path ='/dbfs/sg_claims_files/', baseline_window = 8, L1KPI = 'ALR', L1Num = 'ULT_CLAIM_INCURRED', L1Den = 'ULT_EARNED_PREMIUM', num_function_1 = 'sum', den_function_1= 'sum', L2KPI1 = 'Severity', L2Num1 = 'ULT_CLAIM_INCURRED', L2Den1 = 'CLAIMS_COUNT', sub_num_function_1 = 'sum', sub_den_function_1= 'nunique', L2KPI2 = 'Frequency', L2Num2 = 'CLAIMS_COUNT', L2Den2 = 'AVG_EARNED_EXP_POLCOVG', sub_num_function_2 = 'nunique', sub_den_function_2= 'sum', L2KPI3 = 'Average_Premium', L2Num3 = 'ULT_EARNED_PREMIUM', L2Den3 = 'POL_CVG_COUNT', sub_num_function_3 = 'sum', sub_den_function_3= 'nunique', L2KPI4 = 'EXPOSURE', L2Num4 = 'AVG_EARNED_EXP_POLCOVG', L2Den4= 'POL_CVG_COUNT', sub_num_function_4 = 'sum', sub_den_function_4= 'nunique',ranking_field ='ULT_EARNED_PREMIUM', ranking_function = 'sum', kpi1 = 'ALR',  level1 = 'COVERAGE_GROUP', level2 = 'PROD_CLASS_DESC', level3 = 'GENDER', field1 = 'SPONSOR_NAME', field2 = '', field3 = '', start_date = "'2016-01-01'", end_date = "'2021-12-31'", period = 'quarter', an_date = '2019-01-01', an_item1 = 'MAYBANK', an_item2 = '', an_item3 = '', l1_items = 'IN HOSPITAL CASH', l2_items = 'P/A', bins_cols=['AGE'],bins_size=[3], date_field = 'ACCD_DATE')

#main(file_path ='/dbfs/sg_claims_files/', baseline_window = 8, L1KPI = 'ALR', L1Num = 'ULT_CLAIM_INCURRED', L1Den = 'ULT_EARNED_PREMIUM', num_function_1 = 'sum', den_function_1= 'sum', L2KPI1 = '', L2Num1 = '', L2Den1 = '', sub_num_function_1 = '', sub_den_function_1= '', L2KPI2 = '', L2Num2 = '', L2Den2 = '', sub_num_function_2 = '', sub_den_function_2= '', L2KPI3 = '', L2Num3 = '', L2Den3 = '', sub_num_function_3 = '', sub_den_function_3= '', L2KPI4 = '', L2Num4 = '', L2Den4= '', sub_num_function_4 = '', sub_den_function_4= '',ranking_field ='ULT_EARNED_PREMIUM', ranking_function = 'sum', kpi1 = 'ALR',  level1 = 'COVERAGE_GROUP', level2 = 'PROD_CLASS_DESC', level3 = 'GENDER', field1 = 'SPONSOR_NAME', field2 = '', field3 = '', start_date = "'2016-01-01'", end_date = "'2021-12-31'", period = 'quarter', an_date = '2019-01-01', an_item1 = 'MAYBANK', an_item2 = '', an_item3 = '', l1_items = 'IN HOSPITAL CASH', l2_items = 'P/A', bins_cols=['AGE'],bins_size=[3], date_field = 'ACCD_DATE')

#Two Filter AD Run

main(file_path ='/dbfs/sg_claims_files/', baseline_window = 8, L1KPI = 'ALR', L1Num = 'ULT_CLAIM_INCURRED', L1Den = 'ULT_EARNED_PREMIUM', num_function_1 = 'sum', den_function_1= 'sum', L2KPI1 = 'Severity', L2Num1 = 'ULT_CLAIM_INCURRED', L2Den1 = 'CLAIMS_COUNT', sub_num_function_1 = 'sum', sub_den_function_1= 'nunique', L2KPI2 = 'Frequency', L2Num2 = 'CLAIMS_COUNT', L2Den2 = 'AVG_EARNED_EXP_POLCOVG', sub_num_function_2 = 'nunique', sub_den_function_2= 'sum', L2KPI3 = 'Average_Premium', L2Num3 = 'ULT_EARNED_PREMIUM', L2Den3 = 'POL_CVG_COUNT', sub_num_function_3 = 'sum', sub_den_function_3= 'nunique', L2KPI4 = 'EXPOSURE', L2Num4 = 'AVG_EARNED_EXP_POLCOVG', L2Den4= 'POL_CVG_COUNT', sub_num_function_4 = 'sum', sub_den_function_4= 'nunique',ranking_field ='ULT_EARNED_PREMIUM', ranking_function = 'sum', kpi1 = 'Frequency',  level1 = 'GENDER', level2 = '', level3 = '', field1 = 'SPONSOR_NAME', field2 = 'COVERAGE_GROUP', field3 = '', start_date = "'2016-01-01'", end_date = "'2021-12-31'", period = 'quarter', an_date = '2020-07-01', an_item1 = 'MAYBANK', an_item2 = 'ACCIDENTAL MEDICAL', an_item3 = '', l1_items = 'Female', l2_items = '', bins_cols=['AGE'],bins_size=[3], date_field = 'ACCD_DATE')

#singapore billings database

#main(file_path ='/dbfs/Discovery/SG_BILLING_DATA/SG_Billing.csv', baseline_window = 8, L1KPI = 'PAID RATIO', L1Num = 'Paid_charges', L1Den = 'charge', num_function_1 = 'sum', den_function_1= 'sum', L2KPI1 = '', L2Num1 = '', L2Den1 = '', sub_num_function_1 = '', sub_den_function_1= '', L2KPI2 = '', L2Num2 = '', L2Den2 = '', sub_num_function_2 = '', sub_den_function_2= '', L2KPI3 = '', L2Num3 = '', L2Den3 = '', sub_num_function_3 = '', sub_den_function_3= '', L2KPI4 = '', L2Num4 = '', L2Den4= '', sub_num_function_4 = '', sub_den_function_4= '',ranking_field ='bill_AGE', ranking_function = 'sum', kpi1 = 'PAID RATIO',  level1 = 'PROD_NAME', level2 = 'CHANNEL', level3 = 'BILL_METHOD', field1 = 'SPONSOR_NAME', field2 = '', field3 = '', start_date = "'2016-01-01'", end_date = "'2021-12-31'", period = 'quarter', an_date = '2021-07-01', an_item1 = 'American Express', an_item2 = '', an_item3 = '', l1_items = 'True Vital Recovery', l2_items = 'Telemarketing', bins_cols=['Customer_Age'],bins_size=[3], date_field = 'BILL_DUE_DATE')

0.0367905735814495

timestamp,SPONSOR_NAME,Frequency_Trend,Average_Premium_cont,Severity_cont,Frequency_cont,Average_Premium_%ALR,Severity_%ALR,Frequency_%ALR,Average_Premium_atr,Severity_atr,Frequency_atr,ALR_atr
2020-07-01,MAYBANK,0.0034229530021003983,-0.006035395986229244,0.03138963178437526,0.01578774216710147,-0.04635869566431117,0.24110802178131827,0.12126779021976662,-0.1640473468799734,0.8531976734443314,0.42912465423104773,1.1182749807954058


file read from dbfs started
file read from dbfs ended 23.2562313079834
['2018-10-01', '2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01', '2020-01-01', '2020-04-01', '2020-07-01']

timestamp,GENDER,ALR_Trend,Frequency_Trend_Decomp,Agg_ALR_Trend_Decomp,Frequency_Business_Mix,Frequency_Impact_%Frequency,Frequency_Attribution,Frequency_Att_Business_Mix,Frequency_Att_Agg
2020-07-01T00:00:00.000+0000,Male,0.022374524801112156,0.0030026764496954906,0.045924974828715034,0.033121368398435734,0.0792708725253798,0.08161537473853075,0.9002677907456206,1.0
2020-07-01T00:00:00.000+0000,Female,0.07033421587558879,6.665287333182743E-4,0.045924974828715034,0.033121368398435734,0.009244231042524352,0.018116834515848664,0.9002677907456206,1.0
